In [1]:
import ee
import geemap
import os

credentials_paths = [
    os.path.expanduser('~/.config/earthengine/credentials'),
    os.path.join(os.environ.get('APPDATA', ''), 'earthengine', 'credentials') if os.name == 'nt' else None
]

for cred_path in credentials_paths:
    if cred_path and os.path.exists(cred_path):
        os.remove(cred_path)
        print(f"Cleared existing credentials at {cred_path}")

ee.Authenticate()
ee.Initialize()


Cleared existing credentials at C:\Users\user/.config/earthengine/credentials



Successfully saved authorization token.


In [2]:
dataset = ee.ImageCollection('COPERNICUS/MARINE/SATELLITE_OCEAN_COLOR/V6')

In [3]:

dataset_modis = ee.ImageCollection('MODIS/062/MCD18C2') \
                  .filterDate('2010-01-01', '2010-03-01')

print('Number of images:', dataset_modis.size().getInfo())

dates = dataset_modis.aggregate_array('system:time_start').getInfo()
print('Available dates:', len(dates), 'images')
for date in dates[:5]:  # Show first 5 dates
    print(f"  {ee.Date(date).format('YYYY-MM-dd').getInfo()}")


gmt_1200_par = dataset_modis.select('GMT_1200_PAR')

Map = geemap.Map()
Map.setCenter(6.746, 46.529, 2)

gmt_1200_par_vis = {
    'min': -236,
    'max': 316,
    'palette': ['0f17ff', 'b11406', 'f1ff23']
}

Map.addLayer(
    gmt_1200_par.mean(),
    gmt_1200_par_vis,
    'Total PAR at GMT 12:00 - 2010 (Mean)'
)

Map


Number of images: 59
Available dates: 59 images
  2010-01-01
  2010-01-02
  2010-01-03
  2010-01-04
  2010-01-05


Map(center=[46.529, 6.746], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright…

In [4]:
# ====== CELL 1: Install & Import Libraries ======
# If you are in Google Colab, you might need to install cartopy for mapping:
# !pip install cartopy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from io import StringIO
import geemap
import ee

# For mapping (optional, more complex setup)
# import cartopy.crs as ccrs
# import cartopy.feature as cfeature

print("Libraries imported.")

Libraries imported.


In [5]:
# ====== CELL 2: Fetch and Load the SeaBASS/NOMAD Data ======
# The direct URL to your file
url = "https://seabass.gsfc.nasa.gov/wiki/NOMAD/nomad_seabass_v2.a_2008200.txt"

# Fetch the data
response = requests.get(url)
data_text = response.text

# SeaBASS files have a header ending with `end_header`
header_end = data_text.find('end_header')
header_lines = data_text[:header_end].split('\n')
data_body = data_text[header_end + len('end_header'):].strip()

# Parse header to extract column names and metadata
column_names = []
delimiter = ','
missing_value = '-999'

# Look for /fields= line which contains column names
for line in header_lines:
    if line.startswith('/fields='):
        # Extract column names from /fields= line
        fields = line.replace('/fields=', '').strip()
        column_names = [f.strip() for f in fields.split(',')]
    elif line.startswith('/delimiter='):
        delimiter = line.replace('/delimiter=', '').strip()
    elif line.startswith('/missing='):
        missing_value = line.replace('/missing=', '').strip()

# Print header info for debugging
print("=== NOMAD File Header Information ===")
print(f"Delimiter: {delimiter}")
print(f"Missing value indicator: {missing_value}")
if column_names:
    print(f"Found {len(column_names)} column names in header")
else:
    print("Column names not found in /fields= line, will infer from data")

# Load the main data into a Pandas DataFrame
# Use comma separator as specified in header
df = pd.read_csv(StringIO(data_body), sep=',', na_values=[missing_value, -999], header=None)

# If we found column names in header, use them
if column_names and len(column_names) == len(df.columns):
    df.columns = column_names
    print(f"✓ Using column names from header")
else:
    # NOMAD standard columns (based on typical SeaBASS format)
    # Try to infer from first row or use standard names
    num_cols = len(df.columns)
    print(f"⚠ Column names not in header format. Found {num_cols} columns.")
    print("Using standard NOMAD column names...")
    
    # Standard NOMAD columns (approximate - may need adjustment)
    standard_cols = [
        'year', 'month', 'day', 'hour', 'minute', 'second',
        'lat', 'lon', 'depth', 'temp', 'sal', 'chl', 'a_412', 'a_443', 'a_490',
        'a_510', 'a_532', 'a_555', 'a_670', 'bb_412', 'bb_443', 'bb_490',
        'bb_510', 'bb_532', 'bb_555', 'bb_670', 'Rrs_412', 'Rrs_443', 'Rrs_490',
        'Rrs_510', 'Rrs_532', 'Rrs_555', 'Rrs_670', 'Kd_412', 'Kd_443', 'Kd_490',
        'Kd_510', 'Kd_532', 'Kd_555', 'Kd_670', 'Ed_412', 'Ed_443', 'Ed_490',
        'Ed_510', 'Ed_532', 'Ed_555', 'Ed_670', 'Lu_412', 'Lu_443', 'Lu_490',
        'Lu_510', 'Lu_532', 'Lu_555', 'Lu_670'
    ]
    
    # Use standard names if count matches, otherwise use generic
    if num_cols <= len(standard_cols):
        df.columns = standard_cols[:num_cols]
    else:
        df.columns = [f'col_{i+1}' for i in range(num_cols)]

print(f"\n=== NOMAD Dataset Loaded ===")
print(f"Data shape: {df.shape}")
print(f"Columns ({len(df.columns)}): {list(df.columns)}")
print(f"\nFirst few rows:")
display(df.head())

=== NOMAD File Header Information ===
Delimiter: comma
Missing value indicator: -999
Found 217 column names in header
✓ Using column names from header

=== NOMAD Dataset Loaded ===
Data shape: (4459, 217)
Columns (217): ['year', 'month', 'day', 'hour', 'minute', 'second', 'lat', 'lon', 'id', 'oisst', 'etopo2', 'chl', 'chl_a', 'kd405', 'kd411', 'kd443', 'kd455', 'kd465', 'kd489', 'kd510', 'kd520', 'kd530', 'kd550', 'kd555', 'kd560', 'kd565', 'kd570', 'kd590', 'kd619', 'kd625', 'kd665', 'kd670', 'kd683', 'lw405', 'lw411', 'lw443', 'lw455', 'lw465', 'lw489', 'lw510', 'lw520', 'lw530', 'lw550', 'lw555', 'lw560', 'lw565', 'lw570', 'lw590', 'lw619', 'lw625', 'lw665', 'lw670', 'lw683', 'es405', 'es411', 'es443', 'es455', 'es465', 'es489', 'es510', 'es520', 'es530', 'es550', 'es555', 'es560', 'es565', 'es570', 'es590', 'es619', 'es625', 'es665', 'es670', 'es683', 'ap405', 'ap411', 'ap443', 'ap455', 'ap465', 'ap489', 'ap510', 'ap520', 'ap530', 'ap550', 'ap555', 'ap560', 'ap565', 'ap570', 'ap590

,year,month,day,hour,minute,second,lat,lon,id,oisst,...,allo,diato,lut,zea,chl_b,beta-car,alpha-car,alpha-beta-car,flag,cruise
0,2003,4,15,15,15,0,38.4279,-76.61,1565,3.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20691,ace0301
1,2003,4,15,16,50,0,38.3680,-76.50,1566,3.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20675,ace0301
2,2003,4,15,17,50,0,38.3074,-76.44,1567,3.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20691,ace0301
3,2003,4,17,18,15,0,38.6367,-76.32,1568,3.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20675,ace0301
4,2003,7,21,18,27,0,38.3047,-76.44,1559,22.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20691,ace0302


In [6]:
# Clean the dataset: Remove NaN columns and keep only interesting columns

print("=== Cleaning NOMAD Dataset ===")
print(f"Original shape: {df.shape}")

# Step 1: Remove columns that are entirely NaN
cols_before = len(df.columns)
df = df.dropna(axis=1, how='all')
cols_removed = cols_before - len(df.columns)
print(f"Removed {cols_removed} columns that were entirely NaN")
print(f"Shape after removing NaN columns: {df.shape}")

# Step 2: Define interesting columns to keep
# Keep date/time, location, and key oceanographic/biogeochemical parameters
interesting_cols = []

# Date and time columns
date_time_cols = ['year', 'month', 'day', 'hour', 'minute', 'second']
interesting_cols.extend([col for col in date_time_cols if col in df.columns])

# Location columns
location_cols = ['lat', 'lon', 'id']
interesting_cols.extend([col for col in location_cols if col in df.columns])

# Key oceanographic parameters
key_params = [
    'chl', 'chl_a', 'wt', 'sal', 'poc', 'kpar',  # Basic oceanographic
    'z_37', 'z_10', 'z_01',  # Depth parameters
    'oisst', 'etopo2',  # Environmental
    'chlide_a', 'mv_chl_a', 'dv_chl_a', 'chl_c3', 'chl_c2',  # Chlorophyll variants
]

# Add key spectral bands (select a few representative wavelengths)
# Common ocean color wavelengths: 412, 443, 490, 510, 555, 670 nm
key_wavelengths = ['443', '490', '510', '555', '670']
spectral_prefixes = ['a', 'bb', 'Rrs', 'Kd']  # Absorption, backscatter, remote sensing reflectance, diffuse attenuation

for prefix in spectral_prefixes:
    for wl in key_wavelengths:
        col_name = f'{prefix}{wl}'
        if col_name in df.columns:
            interesting_cols.append(col_name)

# Also keep some specific spectral measurements if they exist
additional_spectral = ['bbr442', 'bbr488', 'bbr510', 'bbr550', 'bbr555', 'bbr671']
interesting_cols.extend([col for col in additional_spectral if col in df.columns])

# Remove duplicates while preserving order
interesting_cols = list(dict.fromkeys(interesting_cols))

# Filter to keep only interesting columns
cols_to_keep = [col for col in interesting_cols if col in df.columns]
df_clean = df[cols_to_keep].copy()

print(f"\nKept {len(cols_to_keep)} interesting columns out of {len(df.columns)} available")
print(f"Final shape: {df_clean.shape}")
print(f"\nColumns kept: {cols_to_keep}")

# Display the cleaned dataset
print(f"\n=== Cleaned NOMAD Dataset ===")
display(df_clean.head(20))

# Update df to the cleaned version
df = df_clean

=== Cleaning NOMAD Dataset ===
Original shape: (4459, 217)
Removed 1 columns that were entirely NaN
Shape after removing NaN columns: (4459, 216)

Kept 23 interesting columns out of 216 available
Final shape: (4459, 23)

Columns kept: ['year', 'month', 'day', 'hour', 'minute', 'second', 'lat', 'lon', 'id', 'a443', 'a510', 'a555', 'a670', 'bb443', 'bb510', 'bb555', 'bb670', 'bbr442', 'bbr488', 'bbr510', 'bbr550', 'bbr555', 'bbr671']

=== Cleaned NOMAD Dataset ===


,year,month,day,hour,minute,second,lat,lon,id,a443,...,bb443,bb510,bb555,bb670,bbr442,bbr488,bbr510,bbr550,bbr555,bbr671
0,2003,4,15,15,15,0,38.4279,-76.6100,1565,2.15391,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2003,4,15,16,50,0,38.3680,-76.5000,1566,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2003,4,15,17,50,0,38.3074,-76.4400,1567,1.27598,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2003,4,17,18,15,0,38.6367,-76.3200,1568,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2003,7,21,18,27,0,38.3047,-76.4400,1559,2.12612,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2003,7,23,15,58,0,38.6360,-76.1600,1562,2.24834,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2003,7,23,17,33,0,38.6417,-76.3200,1563,1.65700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2003,10,13,13,30,0,38.4328,-76.6163,1569,2.60524,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2003,10,13,15,2,0,38.3745,-76.5052,1570,1.78237,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2003,10,16,16,19,0,38.5742,-76.0270,1574,2.01242,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# ====== Visualize Cleaned NOMAD Dataset as a Table ======
print("=== NOMAD: NASA bio-Optical Marine Algorithm Data set ===")
print("Version 2.0 ALPHA, created on 18 July 2008")
print("=" * 80)

# Display dataset information
print(f"\nDataset Summary:")
print(f"  Total records: {len(df):,}")
print(f"  Total columns: {len(df.columns)}")
print(f"  Missing values indicator: -999")

# Show column names and data types
print(f"\n=== Column Information ===")
column_info = pd.DataFrame({
    'Column Name': df.columns,
    'Data Type': df.dtypes.astype(str),
    'Non-Null Count': df.count(),
    'Null Count': df.isnull().sum(),
    'Null Percentage': (df.isnull().sum() / len(df) * 100).round(2)
})
display(column_info.style.format({'Null Percentage': '{:.2f}%'}))

# Display a sample of the data as a formatted table
print(f"\n=== Sample Data (First 30 rows) ===")
display(df.head(30).style.format(precision=4))

# Show basic statistics for numeric columns
print(f"\n=== Basic Statistics for Numeric Columns ===")
numeric_cols = df.select_dtypes(include=[np.number]).columns
if len(numeric_cols) > 0:
    display(df[numeric_cols].describe().style.format(precision=4))
else:
    print("No numeric columns found for statistics")

# Create a date column if year, month, day exist
if all(col in df.columns for col in ['year', 'month', 'day']):
    try:
        df['date'] = pd.to_datetime(df[['year', 'month', 'day']], errors='coerce')
        print(f"\n=== Date Range ===")
        print(f"From: {df['date'].min()}")
        print(f"To: {df['date'].max()}")
        print(f"Total unique dates: {df['date'].nunique()}")
    except:
        pass

print(f"\n=== Full Cleaned Dataset Table (showing first 100 rows) ===")
display(df.head(100).style.format(precision=4))


=== NOMAD: NASA bio-Optical Marine Algorithm Data set ===
Version 2.0 ALPHA, created on 18 July 2008

Dataset Summary:
  Total records: 4,459
  Total columns: 23
  Missing values indicator: -999

=== Column Information ===


,Column Name,Data Type,Non-Null Count,Null Count,Null Percentage
year,year,int64,4459,0,0.00%
month,month,int64,4459,0,0.00%
day,day,int64,4459,0,0.00%
hour,hour,int64,4459,0,0.00%
minute,minute,int64,4459,0,0.00%
second,second,int64,4459,0,0.00%
lat,lat,float64,4459,0,0.00%
lon,lon,float64,4459,0,0.00%
id,id,int64,4459,0,0.00%
a443,a443,float64,1138,3321,74.48%



=== Sample Data (First 30 rows) ===


,year,month,day,hour,minute,second,lat,lon,id,a443,a510,a555,a670,bb443,bb510,bb555,bb670,bbr442,bbr488,bbr510,bbr550,bbr555,bbr671
0,2003,4,15,15,15,0,38.4279,-76.6100,1565,2.1539,0.9276,0.4256,1.0898,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,2003,4,15,16,50,0,38.3680,-76.5000,1566,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,2003,4,15,17,50,0,38.3074,-76.4400,1567,1.2760,0.5603,0.3043,0.8621,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,2003,4,17,18,15,0,38.6367,-76.3200,1568,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,2003,7,21,18,27,0,38.3047,-76.4400,1559,2.1261,0.9713,0.3476,0.9202,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
5,2003,7,23,15,58,0,38.6360,-76.1600,1562,2.2483,0.9333,0.3867,0.8852,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
6,2003,7,23,17,33,0,38.6417,-76.3200,1563,1.6570,0.7738,0.3395,0.8562,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
7,2003,10,13,13,30,0,38.4328,-76.6163,1569,2.6052,1.2909,0.6971,1.3054,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
8,2003,10,13,15,2,0,38.3745,-76.5052,1570,1.7824,0.8223,0.4386,0.9393,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
9,2003,10,16,16,19,0,38.5742,-76.0270,1574,2.0124,0.8268,0.4518,0.6675,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan



=== Basic Statistics for Numeric Columns ===


,year,month,day,hour,minute,second,lat,lon,id,a443,a510,a555,a670,bb443,bb510,bb555,bb670,bbr442,bbr488,bbr510,bbr550,bbr555,bbr671
count,4459.0000,4459.0000,4459.0000,4459.0000,4459.0000,4459.0000,4459.0000,4459.0000,4459.0000,1138.0000,1136.0000,1122.0000,1086.0000,369.0000,369.0000,369.0000,369.0000,247.0000,55.0000,139.0000,33.0000,13.0000,247.0000
mean,1999.2824,5.8388,15.2485,14.9013,27.7589,0.0000,1.8687,-61.5921,4377.3813,0.2897,0.1497,0.1170,0.5039,0.0056,0.0040,0.0034,0.0025,0.0050,0.0031,0.0043,0.0022,0.0029,0.0022
std,3.7691,3.5502,9.0218,4.4719,17.1920,0.0000,44.7651,53.8950,2298.2721,0.4174,0.1817,0.0908,0.1114,0.0019,0.0017,0.0016,0.0013,0.0018,0.0008,0.0019,0.0008,0.0014,0.0014
min,1991.0000,1.0000,1.0000,0.0000,0.0000,0.0000,-77.0356,-179.9550,6.0000,0.0119,0.0342,0.0603,0.4410,0.0027,0.0016,0.0012,0.0007,0.0025,0.0022,0.0017,0.0015,0.0018,0.0008
25%,1997.0000,2.0000,7.0000,13.0000,13.0000,0.0000,-61.2990,-82.7000,2028.5000,0.0479,0.0487,0.0669,0.4467,0.0042,0.0028,0.0022,0.0015,0.0037,0.0025,0.0028,0.0017,0.0021,0.0012
50%,1999.0000,6.0000,15.0000,15.0000,29.0000,0.0000,27.0930,-67.6750,5039.0000,0.0992,0.0697,0.0770,0.4583,0.0050,0.0036,0.0030,0.0022,0.0045,0.0027,0.0036,0.0019,0.0026,0.0018
75%,2002.0000,9.0000,23.0000,18.0000,42.0000,0.0000,34.4585,-63.9615,6271.5000,0.2953,0.1492,0.1211,0.4929,0.0066,0.0049,0.0042,0.0032,0.0058,0.0035,0.0055,0.0021,0.0029,0.0025
max,2007.0000,12.0000,31.0000,23.0000,59.0000,0.0000,79.6900,179.9070,7831.0000,2.6052,1.3521,0.7429,1.3054,0.0118,0.0096,0.0088,0.0075,0.0108,0.0053,0.0100,0.0049,0.0075,0.0080



=== Date Range ===
From: 1991-12-03 00:00:00
To: 2007-09-06 00:00:00
Total unique dates: 1820

=== Full Cleaned Dataset Table (showing first 100 rows) ===


,year,month,day,hour,minute,second,lat,lon,id,a443,a510,a555,a670,bb443,bb510,bb555,bb670,bbr442,bbr488,bbr510,bbr550,bbr555,bbr671,date
0,2003,4,15,15,15,0,38.4279,-76.6100,1565,2.1539,0.9276,0.4256,1.0898,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2003-04-15 00:00:00
1,2003,4,15,16,50,0,38.3680,-76.5000,1566,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2003-04-15 00:00:00
2,2003,4,15,17,50,0,38.3074,-76.4400,1567,1.2760,0.5603,0.3043,0.8621,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2003-04-15 00:00:00
3,2003,4,17,18,15,0,38.6367,-76.3200,1568,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2003-04-17 00:00:00
4,2003,7,21,18,27,0,38.3047,-76.4400,1559,2.1261,0.9713,0.3476,0.9202,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2003-07-21 00:00:00
5,2003,7,23,15,58,0,38.6360,-76.1600,1562,2.2483,0.9333,0.3867,0.8852,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2003-07-23 00:00:00
6,2003,7,23,17,33,0,38.6417,-76.3200,1563,1.6570,0.7738,0.3395,0.8562,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2003-07-23 00:00:00
7,2003,10,13,13,30,0,38.4328,-76.6163,1569,2.6052,1.2909,0.6971,1.3054,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2003-10-13 00:00:00
8,2003,10,13,15,2,0,38.3745,-76.5052,1570,1.7824,0.8223,0.4386,0.9393,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2003-10-13 00:00:00
9,2003,10,16,16,19,0,38.5742,-76.0270,1574,2.0124,0.8268,0.4518,0.6675,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2003-10-16 00:00:00


In [8]:
# ====== Yield Prediction Model (Chlorophyll-a Prediction) ======
# Predict chl_a (chlorophyll-a) as a proxy for ocean productivity/yield
# Based on the same columns from the cleaned dataset

# Install scikit-learn if not available
try:
    import sklearn
    print("✓ scikit-learn is already installed")
except ImportError:
    print("Installing scikit-learn...")
    import sys
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "scikit-learn"])
    print("✓ scikit-learn installed successfully!")
    import sklearn  # Import after installation

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("=== Yield Prediction Model ===")
print("Predicting chlorophyll-a (chl_a) as ocean productivity indicator\n")

# Check if chl_a exists in the dataset
if 'chl_a' not in df.columns:
    print("⚠ Warning: 'chl_a' column not found. Using 'chl' as target instead.")
    target_col = 'chl' if 'chl' in df.columns else None
else:
    target_col = 'chl_a'

if target_col is None:
    print("❌ Error: No suitable target column found for prediction.")
    print("Available columns:", list(df.columns))
else:
    # Prepare features - exclude target and non-numeric columns
    exclude_cols = [target_col, 'year', 'month', 'day', 'hour', 'minute', 'second', 'id', 'date']
    feature_cols = [col for col in df.columns if col not in exclude_cols and df[col].dtype in ['int64', 'float64']]
    
    # Remove features with too many missing values (>50%)
    feature_cols = [col for col in feature_cols if df[col].notna().sum() / len(df) > 0.5]
    
    print(f"Target variable: {target_col}")
    print(f"Number of features: {len(feature_cols)}")
    print(f"Features used: {feature_cols[:10]}..." if len(feature_cols) > 10 else f"Features used: {feature_cols}")
    
    # Prepare data
    X = df[feature_cols].copy()
    y = df[target_col].copy()
    
    # Remove rows where target is missing
    valid_mask = y.notna()
    X = X[valid_mask]
    y = y[valid_mask]
    
    # Fill remaining NaN values in features with median
    X = X.fillna(X.median())
    
    print(f"\nData after cleaning:")
    print(f"  Samples: {len(X)}")
    print(f"  Features: {len(feature_cols)}")
    print(f"  Target range: {y.min():.4f} to {y.max():.4f}")
    print(f"  Target mean: {y.mean():.4f}")
    
    if len(X) > 10:
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Train Random Forest model
        print("\n=== Training Random Forest Model ===")
        rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
        rf_model.fit(X_train, y_train)
        
        # Predictions
        y_train_pred = rf_model.predict(X_train)
        y_test_pred = rf_model.predict(X_test)
        
        # Evaluate
        train_r2 = r2_score(y_train, y_train_pred)
        test_r2 = r2_score(y_test, y_test_pred)
        train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
        train_mae = mean_absolute_error(y_train, y_train_pred)
        test_mae = mean_absolute_error(y_test, y_test_pred)
        
        print(f"\nModel Performance:")
        print(f"  Training R²: {train_r2:.4f}")
        print(f"  Test R²: {test_r2:.4f}")
        print(f"  Training RMSE: {train_rmse:.4f}")
        print(f"  Test RMSE: {test_rmse:.4f}")
        print(f"  Training MAE: {train_mae:.4f}")
        print(f"  Test MAE: {test_mae:.4f}")
        
        # Feature importance
        feature_importance = pd.DataFrame({
            'Feature': feature_cols,
            'Importance': rf_model.feature_importances_
        }).sort_values('Importance', ascending=False)
        
        print(f"\n=== Top 10 Most Important Features ===")
        display(feature_importance.head(10))
        
        # Create predictions dataframe
        predictions_df = pd.DataFrame({
            'Actual': y_test.values,
            'Predicted': y_test_pred,
            'Error': y_test.values - y_test_pred,
            'Abs_Error': np.abs(y_test.values - y_test_pred)
        })
        
        print(f"\n=== Sample Predictions (First 20) ===")
        display(predictions_df.head(20).style.format(precision=4))
        
        # Store model and results
        print(f"\n✓ Model trained successfully!")
        print(f"  Model type: Random Forest Regressor")
        print(f"  Can predict {target_col} based on: {', '.join(feature_cols[:5])}...")
    else:
        print("❌ Error: Not enough data for training (need >10 samples)")


✓ scikit-learn is already installed
=== Yield Prediction Model ===
Predicting chlorophyll-a (chl_a) as ocean productivity indicator

⚠ Warning: 'chl_a' column not found. Using 'chl' as target instead.
❌ Error: No suitable target column found for prediction.
Available columns: ['year', 'month', 'day', 'hour', 'minute', 'second', 'lat', 'lon', 'id', 'a443', 'a510', 'a555', 'a670', 'bb443', 'bb510', 'bb555', 'bb670', 'bbr442', 'bbr488', 'bbr510', 'bbr550', 'bbr555', 'bbr671', 'date']


In [9]:
# ====== Global Marine Phytoplankton Production Dataset ======
# Download and display the dataset from PANGAEA
# Source: https://download.pangaea.de/dataset/932417/files/Global_marine_phytoplankton_production_dataset.txt

import pandas as pd
import numpy as np

url = "https://download.pangaea.de/dataset/932417/files/Global_marine_phytoplankton_production_dataset.txt"

print("=== Downloading Global Marine Phytoplankton Production Dataset ===")
print(f"Source: {url}\n")

# Download the dataset
# The dataset appears to be tab-separated based on the structure
try:
    # Try tab-separated first (most common for PANGAEA datasets)
    df_phytoplankton = pd.read_csv(url, sep='\t', low_memory=False)
    print("✓ Dataset loaded successfully (tab-separated)")
except:
    try:
        # If tab doesn't work, try comma-separated
        df_phytoplankton = pd.read_csv(url, sep=',', low_memory=False)
        print("✓ Dataset loaded successfully (comma-separated)")
    except:
        # Try space-separated as last resort
        df_phytoplankton = pd.read_csv(url, delim_whitespace=True, low_memory=False)
        print("✓ Dataset loaded successfully (space-separated)")

print(f"\nDataset shape: {df_phytoplankton.shape}")
print(f"Columns: {len(df_phytoplankton.columns)}")
print(f"\nColumn names:")
print(list(df_phytoplankton.columns))

# Display basic information
print(f"\n=== Dataset Summary ===")
print(f"Total records: {len(df_phytoplankton):,}")
print(f"Total columns: {len(df_phytoplankton.columns)}")
print(f"\nFirst few rows:")
display(df_phytoplankton.head(20))

# Show data types and null counts
print(f"\n=== Column Information ===")
column_info = pd.DataFrame({
    'Column Name': df_phytoplankton.columns,
    'Data Type': df_phytoplankton.dtypes.astype(str),
    'Non-Null Count': df_phytoplankton.count(),
    'Null Count': df_phytoplankton.isnull().sum(),
    'Null Percentage': (df_phytoplankton.isnull().sum() / len(df_phytoplankton) * 100).round(2)
})
display(column_info.head(30))  # Show first 30 columns

# Show basic statistics for numeric columns
print(f"\n=== Basic Statistics for Numeric Columns ===")
numeric_cols = df_phytoplankton.select_dtypes(include=[np.number]).columns
if len(numeric_cols) > 0:
    print(f"Found {len(numeric_cols)} numeric columns")
    display(df_phytoplankton[numeric_cols].describe().style.format(precision=4))

# Display a sample of the full dataset
print(f"\n=== Sample Data (First 50 rows) ===")
display(df_phytoplankton.head(50).style.format(precision=4))

print(f"\n=== Full Dataset (showing first 100 rows) ===")
display(df_phytoplankton.head(100))


=== Downloading Global Marine Phytoplankton Production Dataset ===
Source: https://download.pangaea.de/dataset/932417/files/Global_marine_phytoplankton_production_dataset.txt

✓ Dataset loaded successfully (tab-separated)

Dataset shape: (37722, 49)
Columns: 49

Column names:
['Count', 'Event', 'Short reference', 'Paper doi', 'Data doi/data link', 'Profile number', 'Date', 'Year', 'Month', 'Day of the year', 'Latitude', 'Longitude', 'Day length (h)', 'Bottom depth (m)', 'Bottom depth sd (m)', 'Mixed Layer Depth (m)', 'Distance from coastline (Km)', 'Euphotic zone depth (m)', 'Sampling depth (m)', 'Max sampling depth (m)', 'Max production depth (m)', 'SST (°C)', 'SST_flag', 'surface PAR (E m^-2 day^-1)', 'PAR_flag', 'Pbopt (mg C mg Chla-1 h-1)', 'Depth-resolved chl a (mg m^-3)', 'Depth-integrated chl a (mg m^-2)', 'Total Chl a (mg m^-2)', 'Depth-resolved primary production (mg C m^-3 day^-1)', 'Depth-integrated primary production (mg C m^-2 day^-1)', 'Production to biomass ratio (mg C d

,Count,Event,Short reference,Paper doi,Data doi/data link,Profile number,Date,Year,Month,Day of the year,...,Max sampling depth magnitude,Max production depth magnitude,SST magnitude,surface PAR magnitude,Pbopt magnitude,Surface chl a magnitude,Depth-integrated chl a magnitude,Total chl a magnitude,Depth-integrated primary production magnitude,Production to biomass ratio magnitude
0,1,Albatros,Behrenfeld and Falkowski 1997\r\n,https://doi.org/10.4319/lo.1997.42.1.0001,http://sites.science.oregonstate.edu/ocean.pro...,1,16/03/1979,1979,3,75,...,very_low,very_low,low,moderate,low,low,low,moderate,low,low
1,2,Albatros,Behrenfeld and Falkowski 1997\r\n,https://doi.org/10.4319/lo.1997.42.1.0001,http://sites.science.oregonstate.edu/ocean.pro...,1,16/03/1979,1979,3,75,...,very_low,very_low,low,moderate,low,low,low,moderate,low,low
2,3,Albatros,Behrenfeld and Falkowski 1997\r\n,https://doi.org/10.4319/lo.1997.42.1.0001,http://sites.science.oregonstate.edu/ocean.pro...,1,16/03/1979,1979,3,75,...,very_low,very_low,low,moderate,low,low,low,moderate,low,low
3,4,Albatros,Behrenfeld and Falkowski 1997\r\n,https://doi.org/10.4319/lo.1997.42.1.0001,http://sites.science.oregonstate.edu/ocean.pro...,1,16/03/1979,1979,3,75,...,very_low,very_low,low,moderate,low,low,low,moderate,low,low
4,5,Albatros,Behrenfeld and Falkowski 1997\r\n,https://doi.org/10.4319/lo.1997.42.1.0001,http://sites.science.oregonstate.edu/ocean.pro...,1,16/03/1979,1979,3,75,...,very_low,very_low,low,moderate,low,low,low,moderate,low,low
5,6,Albatros,Behrenfeld and Falkowski 1997\r\n,https://doi.org/10.4319/lo.1997.42.1.0001,http://sites.science.oregonstate.edu/ocean.pro...,1,16/03/1979,1979,3,75,...,very_low,very_low,low,moderate,low,low,low,moderate,low,low
6,7,Albatros,Behrenfeld and Falkowski 1997\r\n,https://doi.org/10.4319/lo.1997.42.1.0001,http://sites.science.oregonstate.edu/ocean.pro...,2,17/03/1979,1979,3,76,...,very_low,very_low,moderate,moderate,high,low,low,moderate,low,very_low
7,8,Albatros,Behrenfeld and Falkowski 1997\r\n,https://doi.org/10.4319/lo.1997.42.1.0001,http://sites.science.oregonstate.edu/ocean.pro...,2,17/03/1979,1979,3,76,...,very_low,very_low,moderate,moderate,high,low,low,moderate,low,very_low
8,9,Albatros,Behrenfeld and Falkowski 1997\r\n,https://doi.org/10.4319/lo.1997.42.1.0001,http://sites.science.oregonstate.edu/ocean.pro...,2,17/03/1979,1979,3,76,...,very_low,very_low,moderate,moderate,high,low,low,moderate,low,very_low
9,10,Albatros,Behrenfeld and Falkowski 1997\r\n,https://doi.org/10.4319/lo.1997.42.1.0001,http://sites.science.oregonstate.edu/ocean.pro...,2,17/03/1979,1979,3,76,...,very_low,very_low,moderate,moderate,high,low,low,moderate,low,very_low



=== Column Information ===


,Column Name,Data Type,Non-Null Count,Null Count,Null Percentage
Count,Count,int64,37722,0,0.00
Event,Event,object,36872,850,2.25
Short reference,Short reference,object,37722,0,0.00
Paper doi,Paper doi,object,17484,20238,53.65
Data doi/data link,Data doi/data link,object,37722,0,0.00
Profile number,Profile number,int64,37722,0,0.00
Date,Date,object,37722,0,0.00
Year,Year,int64,37722,0,0.00
Month,Month,int64,37722,0,0.00
Day of the year,Day of the year,int64,37722,0,0.00



=== Basic Statistics for Numeric Columns ===
Found 27 numeric columns


,Count,Profile number,Year,Month,Day of the year,Latitude,Longitude,Day length (h),Bottom depth (m),Bottom depth sd (m),Mixed Layer Depth (m),Distance from coastline (Km),Euphotic zone depth (m),Sampling depth (m),Max sampling depth (m),Max production depth (m),SST (°C),SST_flag,surface PAR (E m^-2 day^-1),PAR_flag,Pbopt (mg C mg Chla-1 h-1),Depth-resolved chl a (mg m^-3),Depth-integrated chl a (mg m^-2),Total Chl a (mg m^-2),Depth-resolved primary production (mg C m^-3 day^-1),Depth-integrated primary production (mg C m^-2 day^-1),Production to biomass ratio (mg C day-1 / mg Chl a)
count,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000
mean,18861.5000,2976.9074,1990.4112,5.8772,162.9310,29.5984,-65.8258,12.5998,1758.8310,19.5527,37.9041,178.2200,53.4116,28.9839,60.2525,11.4191,16.7979,0.6964,32.5509,0.5999,5.2990,1.2523,41.7261,35.1436,32.3350,865.7561,28.0695
std,10889.5478,1738.6742,13.1693,3.1722,96.4534,21.6631,75.1616,2.4265,1794.6187,31.6514,46.0709,249.0367,24.1734,35.5830,39.8910,13.7118,6.0477,0.4598,15.4984,0.4899,1.0475,3.0035,73.0256,29.9512,96.9794,1440.2529,58.7901
min,1.0000,1.0000,1958.0000,1.0000,1.0000,-77.7400,-188.5000,7.9336,2.0000,0.0000,1.0000,0.4017,6.6022,0.0000,1.0000,0.0000,-1.7400,0.0000,1.3800,0.0000,1.1300,0.0000,0.0750,5.3676,0.0000,3.3000,0.1464
25%,9431.2500,1431.0000,1980.0000,3.0000,86.0000,21.0625,-120.9217,11.2641,77.0000,2.1140,18.0000,22.1945,33.7132,5.0000,29.5000,2.0000,13.8600,0.0000,14.5100,0.0000,5.1500,0.1600,17.3162,16.4774,1.6000,215.6000,9.0162
50%,18861.5000,2990.0000,1988.0000,5.0000,148.0000,33.3000,-73.5000,12.1034,1146.0000,7.7571,29.0000,97.0917,49.9746,16.3500,49.0000,7.0000,15.0000,1.0000,31.7500,1.0000,5.2700,0.4400,25.9150,26.0146,6.2000,469.5950,17.7586
75%,28291.7500,4404.7500,2002.0000,8.0000,242.0000,40.8000,-17.5000,13.4820,3828.0000,21.3252,46.0000,207.5350,70.2591,40.0000,88.0000,15.0000,19.9200,1.0000,46.8200,1.0000,5.9700,1.2000,43.8750,44.0931,26.2650,963.1500,33.8149
max,37722.0000,6084.0000,2017.0000,12.0000,364.0000,80.7700,172.8013,24.0000,5892.0000,436.8927,524.0000,2078.1300,122.2372,800.0000,250.0000,107.0000,30.1000,1.0000,63.7700,1.0000,6.6300,119.0800,2337.0000,392.2643,3080.0000,30189.7550,3402.6668



=== Sample Data (First 50 rows) ===


,Count,Event,Short reference,Paper doi,Data doi/data link,Profile number,Date,Year,Month,Day of the year,Latitude,Longitude,Day length (h),Bottom depth (m),Bottom depth sd (m),Mixed Layer Depth (m),Distance from coastline (Km),Euphotic zone depth (m),Sampling depth (m),Max sampling depth (m),Max production depth (m),SST (°C),SST_flag,surface PAR (E m^-2 day^-1),PAR_flag,Pbopt (mg C mg Chla-1 h-1),Depth-resolved chl a (mg m^-3),Depth-integrated chl a (mg m^-2),Total Chl a (mg m^-2),Depth-resolved primary production (mg C m^-3 day^-1),Depth-integrated primary production (mg C m^-2 day^-1),Production to biomass ratio (mg C day-1 / mg Chl a),hemisphere,Northern hemisphere season,Bottom depth magnitude,Bottom depth SD magnitude,MLD magnitude,Distance from coastline magnitude,Euphotic zone depth magnitude,Max sampling depth magnitude,Max production depth magnitude,SST magnitude,surface PAR magnitude,Pbopt magnitude,Surface chl a magnitude,Depth-integrated chl a magnitude,Total chl a magnitude,Depth-integrated primary production magnitude,Production to biomass ratio magnitude
0,1,Albatros,Behrenfeld and Falkowski 1997,https://doi.org/10.4319/lo.1997.42.1.0001,http://sites.science.oregonstate.edu/ocean.productivity/data/field/c14/oppwgnew.txt,1,16/03/1979,1979,3,75,38.7800,-72.2700,11.7517,2570,9.9827,35,191.2140,24.7392,0.0000,23.0300,2.5500,7.0000,0,30.8600,0,3.2300,2.7200,69.8231,66.7656,126.0000,1775.3003,25.4257,northern,winter,high,very_low,low,low,very_low,very_low,very_low,low,moderate,low,low,low,moderate,low,low
1,2,Albatros,Behrenfeld and Falkowski 1997,https://doi.org/10.4319/lo.1997.42.1.0001,http://sites.science.oregonstate.edu/ocean.productivity/data/field/c14/oppwgnew.txt,1,16/03/1979,1979,3,75,38.7800,-72.2700,11.7517,2570,9.9827,35,191.2140,24.7392,2.5500,23.0300,2.5500,7.0000,0,30.8600,0,3.2300,2.7800,69.8231,66.7656,165.2400,1775.3003,25.4257,northern,winter,high,very_low,low,low,very_low,very_low,very_low,low,moderate,low,low,low,moderate,low,low
2,3,Albatros,Behrenfeld and Falkowski 1997,https://doi.org/10.4319/lo.1997.42.1.0001,http://sites.science.oregonstate.edu/ocean.productivity/data/field/c14/oppwgnew.txt,1,16/03/1979,1979,3,75,38.7800,-72.2700,11.7517,2570,9.9827,35,191.2140,24.7392,4.5800,23.0300,2.5500,7.0000,0,30.8600,0,3.2300,3.1800,69.8231,66.7656,152.6000,1775.3003,25.4257,northern,winter,high,very_low,low,low,very_low,very_low,very_low,low,moderate,low,low,low,moderate,low,low
3,4,Albatros,Behrenfeld and Falkowski 1997,https://doi.org/10.4319/lo.1997.42.1.0001,http://sites.science.oregonstate.edu/ocean.productivity/data/field/c14/oppwgnew.txt,1,16/03/1979,1979,3,75,38.7800,-72.2700,11.7517,2570,9.9827,35,191.2140,24.7392,9.1600,23.0300,2.5500,7.0000,0,30.8600,0,3.2300,3.2600,69.8231,66.7656,94.6800,1775.3003,25.4257,northern,winter,high,very_low,low,low,very_low,very_low,very_low,low,moderate,low,low,low,moderate,low,low
4,5,Albatros,Behrenfeld and Falkowski 1997,https://doi.org/10.4319/lo.1997.42.1.0001,http://sites.science.oregonstate.edu/ocean.productivity/data/field/c14/oppwgnew.txt,1,16/03/1979,1979,3,75,38.7800,-72.2700,11.7517,2570,9.9827,35,191.2140,24.7392,14.9800,23.0300,2.5500,7.0000,0,30.8600,0,3.2300,3.1000,69.8231,66.7656,30.5000,1775.3003,25.4257,northern,winter,high,very_low,low,low,very_low,very_low,very_low,low,moderate,low,low,low,moderate,low,low
5,6,Albatros,Behrenfeld and Falkowski 1997,https://doi.org/10.4319/lo.1997.42.1.0001,http://sites.science.oregonstate.edu/ocean.productivity/data/field/c14/oppwgnew.txt,1,16/03/1979,1979,3,75,38.7800,-72.2700,11.7517,2570,9.9827,35,191.2140,24.7392,23.0300,23.0300,2.5500,7.0000,0,30.8600,0,3.2300,2.7400,69.8231,66.7656,6.9700,1775.3003,25.4257,northern,winter,high,very_low,low,low,very_low,very_low,very_low,low,moderate,low,low,low,moderate,low,low
6,7,Albatros,Behrenfeld and Falkowski 1997,https://doi.org/10.4319/lo.1997.42.1.0001,http://sites.science.oregonstate.edu/ocean.productivity/data/field/c14/oppwgnew.txt,2,17/03/1979,1979,3,76,


=== Full Dataset (showing first 100 rows) ===


,Count,Event,Short reference,Paper doi,Data doi/data link,Profile number,Date,Year,Month,Day of the year,...,Max sampling depth magnitude,Max production depth magnitude,SST magnitude,surface PAR magnitude,Pbopt magnitude,Surface chl a magnitude,Depth-integrated chl a magnitude,Total chl a magnitude,Depth-integrated primary production magnitude,Production to biomass ratio magnitude
0,1,Albatros,Behrenfeld and Falkowski 1997\r\n,https://doi.org/10.4319/lo.1997.42.1.0001,http://sites.science.oregonstate.edu/ocean.pro...,1,16/03/1979,1979,3,75,...,very_low,very_low,low,moderate,low,low,low,moderate,low,low
1,2,Albatros,Behrenfeld and Falkowski 1997\r\n,https://doi.org/10.4319/lo.1997.42.1.0001,http://sites.science.oregonstate.edu/ocean.pro...,1,16/03/1979,1979,3,75,...,very_low,very_low,low,moderate,low,low,low,moderate,low,low
2,3,Albatros,Behrenfeld and Falkowski 1997\r\n,https://doi.org/10.4319/lo.1997.42.1.0001,http://sites.science.oregonstate.edu/ocean.pro...,1,16/03/1979,1979,3,75,...,very_low,very_low,low,moderate,low,low,low,moderate,low,low
3,4,Albatros,Behrenfeld and Falkowski 1997\r\n,https://doi.org/10.4319/lo.1997.42.1.0001,http://sites.science.oregonstate.edu/ocean.pro...,1,16/03/1979,1979,3,75,...,very_low,very_low,low,moderate,low,low,low,moderate,low,low
4,5,Albatros,Behrenfeld and Falkowski 1997\r\n,https://doi.org/10.4319/lo.1997.42.1.0001,http://sites.science.oregonstate.edu/ocean.pro...,1,16/03/1979,1979,3,75,...,very_low,very_low,low,moderate,low,low,low,moderate,low,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,Argus,Behrenfeld and Falkowski 1997\r\n,https://doi.org/10.4319/lo.1997.42.1.0001,http://sites.science.oregonstate.edu/ocean.pro...,18,03/04/1978,1978,4,93,...,very_low,very_low,low,high,low,very_low,high,moderate,low,very_low
96,97,Argus,Behrenfeld and Falkowski 1997\r\n,https://doi.org/10.4319/lo.1997.42.1.0001,http://sites.science.oregonstate.edu/ocean.pro...,18,03/04/1978,1978,4,93,...,very_low,very_low,low,high,low,very_low,high,moderate,low,very_low
97,98,Argus,Behrenfeld and Falkowski 1997\r\n,https://doi.org/10.4319/lo.1997.42.1.0001,http://sites.science.oregonstate.edu/ocean.pro...,19,04/04/1978,1978,4,94,...,very_low,high,low,high,low,very_low,moderate,moderate,low,very_low
98,99,Argus,Behrenfeld and Falkowski 1997\r\n,https://doi.org/10.4319/lo.1997.42.1.0001,http://sites.science.oregonstate.edu/ocean.pro...,19,04/04/1978,1978,4,94,...,very_low,high,low,high,low,very_low,moderate,moderate,low,very_low


In [10]:
# ====== Display Most Recent Rows - Key Algae Production Columns ======
# Order by date (most recent first) and show only 20 rows with production-affecting columns

print("=== Most Recent Algae Production Data ===")
print("Ordered by date (most recent first), showing 20 rows\n")

# Select the most important columns affecting algae production
production_key_cols = []

# Date/Time columns
date_cols = ['Date', 'Year', 'Month', 'Day of the year']
for col in date_cols:
    if col in df_phytoplankton.columns:
        production_key_cols.append(col)

# Location
location_cols = ['Latitude', 'Longitude']
for col in location_cols:
    if col in df_phytoplankton.columns:
        production_key_cols.append(col)

# Environmental factors critical for algae growth
env_cols = [
    'SST (°C)',  # Sea Surface Temperature - critical for growth
    'surface PAR (E m^-2 day^-1)',  # Photosynthetically Active Radiation - energy for photosynthesis
    'Euphotic zone depth (m)',  # Light penetration depth
    'Mixed Layer Depth (m)',  # Mixing affects nutrient availability
    'Sampling depth (m)',  # Where measurement was taken
]

for col in env_cols:
    if col in df_phytoplankton.columns:
        production_key_cols.append(col)

# Chlorophyll a - biomass indicator
chl_cols = [
    'Depth-resolved chl a (mg m^-3)',
    'Depth-integrated chl a (mg m^-2)',
    'Total Chl a (mg m^-2)',
]

for col in chl_cols:
    if col in df_phytoplankton.columns:
        production_key_cols.append(col)

# Primary production - actual productivity
production_cols = [
    'Depth-resolved primary production (mg C m^-3 day^-1)',
    'Depth-integrated primary production (mg C m^-2 day^-1)',
    'Production to biomass ratio (mg C day-1 / mg Chl a)',
    'Pbopt (mg C mg Chla-1 h-1)',  # Maximum production rate
]

for col in production_cols:
    if col in df_phytoplankton.columns:
        production_key_cols.append(col)

# Remove duplicates while preserving order
production_key_cols = list(dict.fromkeys(production_key_cols))

print(f"Selected {len(production_key_cols)} key columns for algae production analysis")
print(f"Columns: {production_key_cols}\n")

# Create filtered dataframe with key columns
df_production = df_phytoplankton[production_key_cols].copy()

# Create a date column for sorting
if all(col in df_production.columns for col in ['Year', 'Month']):
    # Try to create date from Year, Month, and Day of the year if available
    if 'Day of the year' in df_production.columns:
        # Convert day of year to actual date
        df_production['Date_parsed'] = pd.to_datetime(
            df_production['Year'].astype(str) + '-' + 
            df_production['Month'].astype(str).str.zfill(2) + '-01',
            errors='coerce'
        )
        # Add days to get approximate date
        df_production['Date_parsed'] = df_production['Date_parsed'] + pd.to_timedelta(
            (df_production['Day of the year'] - 1).fillna(0), unit='D'
        )
    elif 'Date' in df_production.columns:
        df_production['Date_parsed'] = pd.to_datetime(df_production['Date'], errors='coerce')
    else:
        # Use Year and Month only
        df_production['Date_parsed'] = pd.to_datetime(
            df_production['Year'].astype(str) + '-' + 
            df_production['Month'].astype(str).str.zfill(2) + '-01',
            errors='coerce'
        )
    
    # Sort by date (most recent first)
    df_production = df_production.sort_values('Date_parsed', ascending=False, na_position='last')
    
    # Remove the temporary date column for display
    if 'Date_parsed' in df_production.columns:
        df_production_display = df_production.drop('Date_parsed', axis=1)
    else:
        df_production_display = df_production.copy()
else:
    # Fallback: sort by Year, Month if Date parsing fails
    sort_cols = []
    if 'Year' in df_production.columns:
        sort_cols.append('Year')
    if 'Month' in df_production.columns:
        sort_cols.append('Month')
    if sort_cols:
        df_production = df_production.sort_values(sort_cols, ascending=False, na_position='last')
    df_production_display = df_production.copy()

# Display the 20 most recent rows
print("=== 20 Most Recent Records (Ordered by Date) ===")
print(f"Showing most recent data affecting algae production\n")
display(df_production_display.head(20).style.format(precision=4))

# Show date range
if 'Date_parsed' in df_production.columns:
    print(f"\nDate range in dataset:")
    print(f"  Most recent: {df_production['Date_parsed'].max()}")
    print(f"  Oldest: {df_production['Date_parsed'].min()}")
elif 'Year' in df_production.columns:
    print(f"\nYear range in dataset:")
    print(f"  Most recent year: {df_production['Year'].max()}")
    print(f"  Oldest year: {df_production['Year'].min()}")


=== Most Recent Algae Production Data ===
Ordered by date (most recent first), showing 20 rows

Selected 18 key columns for algae production analysis
Columns: ['Date', 'Year', 'Month', 'Day of the year', 'Latitude', 'Longitude', 'SST (°C)', 'surface PAR (E m^-2 day^-1)', 'Euphotic zone depth (m)', 'Mixed Layer Depth (m)', 'Sampling depth (m)', 'Depth-resolved chl a (mg m^-3)', 'Depth-integrated chl a (mg m^-2)', 'Total Chl a (mg m^-2)', 'Depth-resolved primary production (mg C m^-3 day^-1)', 'Depth-integrated primary production (mg C m^-2 day^-1)', 'Production to biomass ratio (mg C day-1 / mg Chl a)', 'Pbopt (mg C mg Chla-1 h-1)']

=== 20 Most Recent Records (Ordered by Date) ===
Showing most recent data affecting algae production



,Date,Year,Month,Day of the year,Latitude,Longitude,SST (°C),surface PAR (E m^-2 day^-1),Euphotic zone depth (m),Mixed Layer Depth (m),Sampling depth (m),Depth-resolved chl a (mg m^-3),Depth-integrated chl a (mg m^-2),Total Chl a (mg m^-2),Depth-resolved primary production (mg C m^-3 day^-1),Depth-integrated primary production (mg C m^-2 day^-1),Production to biomass ratio (mg C day-1 / mg Chl a),Pbopt (mg C mg Chla-1 h-1)
14836,15/08/2017,2017,8,227,34.2733,-120.0183,19.0000,57.5100,18.1598,21,2.0000,6.1600,108.1550,101.0517,172.7000,1666.4100,15.4076,6.6000
14841,15/08/2017,2017,8,227,34.2733,-120.0183,19.0000,57.5100,18.1598,21,23.0000,0.7900,108.1550,101.0517,0.6300,1666.4100,15.4076,6.6000
14838,15/08/2017,2017,8,227,34.2733,-120.0183,19.0000,57.5100,18.1598,21,6.0000,10.4600,108.1550,101.0517,156.3000,1666.4100,15.4076,6.6000
14839,15/08/2017,2017,8,227,34.2733,-120.0183,19.0000,57.5100,18.1598,21,10.0000,4.7500,108.1550,101.0517,41.3000,1666.4100,15.4076,6.6000
14840,15/08/2017,2017,8,227,34.2733,-120.0183,19.0000,57.5100,18.1598,21,19.0000,1.3400,108.1550,101.0517,3.4000,1666.4100,15.4076,6.6000
14837,15/08/2017,2017,8,227,34.2733,-120.0183,19.0000,57.5100,18.1598,21,5.0000,9.6400,108.1550,101.0517,189.7000,1666.4100,15.4076,6.6000
14824,14/08/2017,2017,8,226,35.0883,-120.7750,15.0600,43.4400,16.3429,20,18.0000,1.4800,126.5400,116.3892,2.4000,2234.5000,17.6584,5.7000
14823,14/08/2017,2017,8,226,35.0883,-120.7750,15.0600,43.4400,16.3429,20,10.0000,8.9500,126.5400,116.3892,93.2000,2234.5000,17.6584,5.7000
14822,14/08/2017,2017,8,226,35.0883,-120.7750,15.0600,43.4400,16.3429,20,5.0000,8.7300,126.5400,116.3892,202.7000,2234.5000,17.6584,5.7000
14821,14/08/2017,2017,8,226,35.0883,-120.7750,15.0600,43.4400,16.3429,20,4.0000,7.8900,126.5400,116.3892,232.2000,2234.5000,17.6584,5.7000



Date range in dataset:
  Most recent: 2018-03-15 00:00:00
  Oldest: 1958-11-04 00:00:00


In [11]:
# ====== Comprehensive Data Cleaning for Phytoplankton Dataset ======
# Clean, transform, and save the dataset

import pandas as pd
import numpy as np
from datetime import datetime

print("=== Starting Comprehensive Data Cleaning ===\n")

# Start with the original dataset
df_clean = df_phytoplankton.copy()

print(f"Original dataset shape: {df_clean.shape}")

# ====== STEP 1: Create unified date column ======
print("\n1. Creating unified date column...")
if all(col in df_clean.columns for col in ['Year', 'Month']):
    # Create date from Year, Month, and Day of the year
    if 'Day of the year' in df_clean.columns:
        # Convert day of year to actual day/month
        df_clean['date'] = pd.to_datetime(
            df_clean['Year'].astype(str) + '-' + 
            df_clean['Month'].astype(str).str.zfill(2) + '-01',
            errors='coerce'
        )
        # Add days from day of year
        df_clean['date'] = df_clean['date'] + pd.to_timedelta(
            (df_clean['Day of the year'] - 1).fillna(0), unit='D'
        )
    elif 'Date' in df_clean.columns:
        df_clean['date'] = pd.to_datetime(df_clean['Date'], errors='coerce')
    else:
        # Use Year and Month only (set to first day of month)
        df_clean['date'] = pd.to_datetime(
            df_clean['Year'].astype(str) + '-' + 
            df_clean['Month'].astype(str).str.zfill(2) + '-01',
            errors='coerce'
        )
    
    # Remove old date columns
    date_cols_to_remove = ['Date', 'Year', 'Month', 'Day of the year']
    for col in date_cols_to_remove:
        if col in df_clean.columns:
            df_clean = df_clean.drop(col, axis=1)
    print(f"   ✓ Created unified 'date' column")
else:
    print("   ⚠ Warning: Could not create date column - missing Year/Month")

# ====== STEP 2: Rename columns to more logical names ======
print("\n2. Renaming columns to logical names...")

# Create a mapping dictionary for column renaming
column_rename_map = {
    # Location
    'Latitude': 'latitude',
    'Longitude': 'longitude',
    
    # Environmental parameters
    'SST (°C)': 'sea_surface_temp_c',
    'SST_flag': 'sst_quality_flag',
    'surface PAR (E m^-2 day^-1)': 'surface_par',
    'PAR_flag': 'par_quality_flag',
    'Day length (h)': 'day_length_hours',
    
    # Depth measurements
    'Bottom depth (m)': 'bottom_depth_m',
    'Bottom depth sd (m)': 'bottom_depth_sd_m',
    'Mixed Layer Depth (m)': 'mixed_layer_depth_m',
    'Euphotic zone depth (m)': 'euphotic_zone_depth_m',
    'Sampling depth (m)': 'sampling_depth_m',
    'Max sampling depth (m)': 'max_sampling_depth_m',
    'Max production depth (m)': 'max_production_depth_m',
    'Distance from coastline (Km)': 'distance_from_coast_km',
    
    # Chlorophyll measurements
    'Depth-resolved chl a (mg m^-3)': 'chl_a_resolved_mg_m3',
    'Depth-integrated chl a (mg m^-2)': 'chl_a_integrated_mg_m2',
    'Total Chl a (mg m^-2)': 'chl_a_total_mg_m2',
    
    # Primary production
    'Depth-resolved primary production (mg C m^-3 day^-1)': 'primary_prod_resolved_mg_c_m3_day',
    'Depth-integrated primary production (mg C m^-2 day^-1)': 'primary_prod_integrated_mg_c_m2_day',
    'Production to biomass ratio (mg C day-1 / mg Chl a)': 'prod_to_biomass_ratio',
    'Pbopt (mg C mg Chla-1 h-1)': 'max_production_rate_pbopt',
    
    # Other identifiers
    'Event': 'event_name',
    'Short reference': 'reference',
    'Paper doi': 'paper_doi',
    'Data doi/data link': 'data_link',
    'Profile number': 'profile_number',
    'Count': 'record_id',
    
    # Magnitude categories (if they exist)
    'hemisphere': 'hemisphere',
    'Northern hemisphere season': 'season',
}

# Apply renaming (only for columns that exist)
columns_to_rename = {k: v for k, v in column_rename_map.items() if k in df_clean.columns}
df_clean = df_clean.rename(columns=columns_to_rename)
print(f"   ✓ Renamed {len(columns_to_rename)} columns")

# ====== STEP 3: Remove unnecessary columns ======
print("\n3. Removing unnecessary columns...")
cols_before = len(df_clean.columns)

# Remove columns that are mostly flags or metadata we don't need
unnecessary_cols = [
    'id',  # If exists and redundant
    'record_id',  # Keep only if needed, but usually redundant with index
]

# Remove magnitude category columns (they're derived, not original data)
magnitude_cols = [col for col in df_clean.columns if 'magnitude' in col.lower()]
unnecessary_cols.extend(magnitude_cols)

# Remove quality flags if they're not critical (keep only if they indicate data quality issues)
# Actually, let's keep quality flags as they're useful

# Remove columns that are entirely empty or have very low data coverage (<5%)
for col in df_clean.columns:
    if df_clean[col].notna().sum() / len(df_clean) < 0.05:
        unnecessary_cols.append(col)

# Remove duplicates from list
unnecessary_cols = list(set([col for col in unnecessary_cols if col in df_clean.columns]))
df_clean = df_clean.drop(columns=unnecessary_cols, errors='ignore')
cols_removed = cols_before - len(df_clean.columns)
print(f"   ✓ Removed {cols_removed} unnecessary columns")

# ====== STEP 4: Clean null values ======
print("\n4. Cleaning null values...")
nulls_before = df_clean.isnull().sum().sum()

# Strategy: 
# - For numeric columns: fill with median (less sensitive to outliers than mean)
# - For categorical: fill with mode or 'unknown'
# - For critical columns: keep nulls or use forward fill if time series

# Fill numeric columns with median
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if col != 'date':  # Don't fill date column
        median_val = df_clean[col].median()
        if pd.notna(median_val):
            df_clean[col] = df_clean[col].fillna(median_val)

# Fill categorical columns with mode or 'unknown'
categorical_cols = df_clean.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if col != 'date':
        mode_val = df_clean[col].mode()
        if len(mode_val) > 0:
            df_clean[col] = df_clean[col].fillna(mode_val[0])
        else:
            df_clean[col] = df_clean[col].fillna('unknown')

nulls_after = df_clean.isnull().sum().sum()
nulls_cleaned = nulls_before - nulls_after
print(f"   ✓ Cleaned {nulls_cleaned:,} null values ({nulls_before:,} → {nulls_after:,})")

# ====== STEP 5: Remove duplicate rows ======
print("\n5. Removing duplicate rows...")
rows_before = len(df_clean)
df_clean = df_clean.drop_duplicates()
rows_removed = rows_before - len(df_clean)
print(f"   ✓ Removed {rows_removed} duplicate rows")

# ====== STEP 6: Remove rows with critical missing data ======
print("\n6. Removing rows with critical missing data...")
rows_before = len(df_clean)

# Remove rows where essential columns are missing
critical_cols = ['latitude', 'longitude', 'date']
if all(col in df_clean.columns for col in critical_cols):
    df_clean = df_clean.dropna(subset=critical_cols)
    rows_removed = rows_before - len(df_clean)
    print(f"   ✓ Removed {rows_removed} rows with missing critical data (location/date)")

# ====== STEP 7: Sort by date ======
print("\n7. Sorting by date...")
if 'date' in df_clean.columns:
    df_clean = df_clean.sort_values('date', ascending=False, na_position='last')
    print(f"   ✓ Sorted by date (most recent first)")

# ====== STEP 8: Reset index ======
df_clean = df_clean.reset_index(drop=True)

# ====== STEP 9: Final data quality check ======
print("\n8. Final data quality check...")
print(f"   Final shape: {df_clean.shape}")
print(f"   Columns: {len(df_clean.columns)}")
print(f"   Rows: {len(df_clean):,}")
print(f"   Remaining nulls: {df_clean.isnull().sum().sum():,}")

# Show column list
print(f"\n   Cleaned columns ({len(df_clean.columns)}):")
for i, col in enumerate(df_clean.columns, 1):
    null_pct = (df_clean[col].isnull().sum() / len(df_clean) * 100)
    print(f"   {i:2d}. {col:40s} (nulls: {null_pct:.1f}%)")

# ====== STEP 10: Save to CSV ======
print("\n9. Saving cleaned dataset to CSV...")
output_filename = 'phytoplankton_production_cleaned.csv'
df_clean.to_csv(output_filename, index=False, encoding='utf-8')

# Get file size
import os
file_size = os.path.getsize(output_filename)
file_size_mb = file_size / (1024 * 1024)
print(f"   ✓ Saved to: {output_filename}")
print(f"   File size: {file_size_mb:.2f} MB ({file_size:,} bytes)")

# Display sample of cleaned data
print(f"\n=== Sample of Cleaned Data (First 10 rows) ===")
display(df_clean.head(10))

print(f"\n=== Data Cleaning Complete! ===")
print(f"Original: {df_phytoplankton.shape[0]:,} rows × {df_phytoplankton.shape[1]} columns")
print(f"Cleaned:  {df_clean.shape[0]:,} rows × {df_clean.shape[1]} columns")
print(f"Saved to: {output_filename}")


=== Starting Comprehensive Data Cleaning ===

Original dataset shape: (37722, 49)

1. Creating unified date column...
   ✓ Created unified 'date' column

2. Renaming columns to logical names...
   ✓ Renamed 30 columns

3. Removing unnecessary columns...
   ✓ Removed 16 unnecessary columns

4. Cleaning null values...
   ✓ Cleaned 21,088 null values (21,088 → 0)

5. Removing duplicate rows...
   ✓ Removed 0 duplicate rows

6. Removing rows with critical missing data...
   ✓ Removed 0 rows with missing critical data (location/date)

7. Sorting by date...
   ✓ Sorted by date (most recent first)

8. Final data quality check...
   Final shape: (37722, 30)
   Columns: 30
   Rows: 37,722
   Remaining nulls: 0

   Cleaned columns (30):
    1. event_name                               (nulls: 0.0%)
    2. reference                                (nulls: 0.0%)
    3. paper_doi                                (nulls: 0.0%)
    4. data_link                                (nulls: 0.0%)
    5. profile_

,event_name,reference,paper_doi,data_link,profile_number,latitude,longitude,day_length_hours,bottom_depth_m,bottom_depth_sd_m,...,max_production_rate_pbopt,chl_a_resolved_mg_m3,chl_a_integrated_mg_m2,chl_a_total_mg_m2,primary_prod_resolved_mg_c_m3_day,primary_prod_integrated_mg_c_m2_day,prod_to_biomass_ratio,hemisphere,season,date
0,CALCOFI,calcofi.org,https://doi.org/10.4319/lo.1997.42.1.0001,http://calcofi.org/downloads/IEH/,2309,34.273333,-120.018333,13.300491,582,2.249829,...,6.6,6.16,108.155,101.051665,172.70,1666.41,15.407609,northern,summer,2018-03-15
1,CALCOFI,calcofi.org,https://doi.org/10.4319/lo.1997.42.1.0001,http://calcofi.org/downloads/IEH/,2309,34.273333,-120.018333,13.300491,582,2.249829,...,6.6,0.79,108.155,101.051665,0.63,1666.41,15.407609,northern,summer,2018-03-15
2,CALCOFI,calcofi.org,https://doi.org/10.4319/lo.1997.42.1.0001,http://calcofi.org/downloads/IEH/,2309,34.273333,-120.018333,13.300491,582,2.249829,...,6.6,10.46,108.155,101.051665,156.30,1666.41,15.407609,northern,summer,2018-03-15
3,CALCOFI,calcofi.org,https://doi.org/10.4319/lo.1997.42.1.0001,http://calcofi.org/downloads/IEH/,2309,34.273333,-120.018333,13.300491,582,2.249829,...,6.6,4.75,108.155,101.051665,41.30,1666.41,15.407609,northern,summer,2018-03-15
4,CALCOFI,calcofi.org,https://doi.org/10.4319/lo.1997.42.1.0001,http://calcofi.org/downloads/IEH/,2309,34.273333,-120.018333,13.300491,582,2.249829,...,6.6,1.34,108.155,101.051665,3.40,1666.41,15.407609,northern,summer,2018-03-15
5,CALCOFI,calcofi.org,https://doi.org/10.4319/lo.1997.42.1.0001,http://calcofi.org/downloads/IEH/,2309,34.273333,-120.018333,13.300491,582,2.249829,...,6.6,9.64,108.155,101.051665,189.70,1666.41,15.407609,northern,summer,2018-03-15
6,CALCOFI,calcofi.org,https://doi.org/10.4319/lo.1997.42.1.0001,http://calcofi.org/downloads/IEH/,2306,35.088333,-120.775000,13.373152,73,4.012327,...,5.7,1.48,126.540,116.389185,2.40,2234.50,17.658448,northern,summer,2018-03-14
7,CALCOFI,calcofi.org,https://doi.org/10.4319/lo.1997.42.1.0001,http://calcofi.org/downloads/IEH/,2306,35.088333,-120.775000,13.373152,73,4.012327,...,5.7,8.95,126.540,116.389185,93.20,2234.50,17.658448,northern,summer,2018-03-14
8,CALCOFI,calcofi.org,https://doi.org/10.4319/lo.1997.42.1.0001,http://calcofi.org/downloads/IEH/,2306,35.088333,-120.775000,13.373152,73,4.012327,...,5.7,8.73,126.540,116.389185,202.70,2234.50,17.658448,northern,summer,2018-03-14
9,CALCOFI,calcofi.org,https://doi.org/10.4319/lo.1997.42.1.0001,http://calcofi.org/downloads/IEH/,2306,35.088333,-120.775000,13.373152,73,4.012327,...,5.7,7.89,126.540,116.389185,232.20,2234.50,17.658448,northern,summer,2018-03-14



=== Data Cleaning Complete! ===
Original: 37,722 rows × 49 columns
Cleaned:  37,722 rows × 30 columns
Saved to: phytoplankton_production_cleaned.csv


In [12]:
# ====== Display Most Recent Rows - Key Algae Production Columns ======
# Filter to show only the most interesting columns affecting phytoplankton/algae production
# Sort by most recent dates first

print("=== Most Recent Data - Key Algae Production Factors ===\n")

# Identify key columns for algae production
production_key_cols = []

# Date/Time columns (for sorting and context)
date_cols = ['Date', 'Year', 'Month', 'Day of the year']
for col in date_cols:
    if col in df_phytoplankton.columns:
        production_key_cols.append(col)

# Location
location_cols = ['Latitude', 'Longitude']
for col in location_cols:
    if col in df_phytoplankton.columns:
        production_key_cols.append(col)

# Environmental factors critical for algae growth
env_cols = [
    'SST (°C)',  # Sea Surface Temperature
    'surface PAR (E m^-2 day^-1)',  # Photosynthetically Active Radiation
    'Day length (h)',  # Daylight hours
    'Mixed Layer Depth (m)',  # Affects nutrient availability
    'Euphotic zone depth (m)',  # Light penetration depth
    'Sampling depth (m)',  # Where measurement was taken
]

# Chlorophyll a measurements (biomass indicator)
chl_cols = [
    'Depth-resolved chl a (mg m^-3)',
    'Depth-integrated chl a (mg m^-2)',
    'Total Chl a (mg m^-2)',
]

# Primary production measurements (actual productivity)
production_cols = [
    'Pbopt (mg C mg Chla-1 h-1)',  # Maximum production rate
    'Depth-resolved primary production (mg C m^-3 day^-1)',
    'Depth-integrated primary production (mg C m^-2 day^-1)',
    'Production to biomass ratio (mg C day-1 / mg Chl a)',
]

# Additional important factors
other_key_cols = [
    'Bottom depth (m)',
    'Distance from coastline (Km)',
    'Max sampling depth (m)',
    'Max production depth (m)',
]

# Combine all relevant columns
all_key_cols = date_cols + location_cols + env_cols + chl_cols + production_cols + other_key_cols

# Filter to only columns that exist in the dataset
production_key_cols = [col for col in all_key_cols if col in df_phytoplankton.columns]

# Remove duplicates while preserving order
production_key_cols = list(dict.fromkeys(production_key_cols))

print(f"Selected {len(production_key_cols)} key columns for algae production analysis")
print(f"Columns: {production_key_cols}\n")

# Create filtered dataframe with only key columns
df_production = df_phytoplankton[production_key_cols].copy()

# Sort by Year (most recent first), then Month, then Day of the year
sort_cols = []
if 'Year' in df_production.columns:
    sort_cols.append('Year')
if 'Month' in df_production.columns:
    sort_cols.append('Month')
if 'Day of the year' in df_production.columns:
    sort_cols.append('Day of the year')

if sort_cols:
    # Sort descending by Year (most recent first), then by Month and Day
    df_production = df_production.sort_values(
        by=sort_cols, 
        ascending=[False] * len(sort_cols),  # All descending (most recent first)
        na_position='last'
    )
    print(f"Sorted by: {sort_cols} (most recent year first)")
else:
    print("⚠ Warning: No date columns found for sorting. Showing data as-is.")

# Remove rows where all key production values are missing
# Keep rows that have at least one production-related value
production_value_cols = [col for col in production_key_cols if any(term in col.lower() for term in ['production', 'chl', 'par', 'sst'])]
if production_value_cols:
    df_production = df_production.dropna(subset=production_value_cols, how='all')

print(f"\nFiltered dataset shape: {df_production.shape}")

# Get the first 50 rows (which are the most recent after sorting descending)
# Since we sorted descending (most recent first), the newest data is at the top
most_recent_50 = df_production.head(500).copy()

# Show year range
if 'Year' in most_recent_50.columns:
    years = most_recent_50['Year'].dropna().unique()
    if len(years) > 0:
        print(f"Year range in most recent 50 rows: {int(years.min())} to {int(years.max())}")
        print(f"Most recent year: {int(years.max())}")

print(f"\n=== Last 50 Rows - Sorted by Year (Most Recent First) ===")
print("Showing the 50 most recent records, starting from the most recent year\n")
display(most_recent_50.style.format(precision=4))

# Show summary statistics for the key columns
print(f"\n=== Summary Statistics for Key Production Columns ===")
numeric_prod_cols = df_production.select_dtypes(include=[np.number]).columns
if len(numeric_prod_cols) > 0:
    display(df_production[numeric_prod_cols].describe().style.format(precision=4))


=== Most Recent Data - Key Algae Production Factors ===

Selected 23 key columns for algae production analysis
Columns: ['Date', 'Year', 'Month', 'Day of the year', 'Latitude', 'Longitude', 'SST (°C)', 'surface PAR (E m^-2 day^-1)', 'Day length (h)', 'Mixed Layer Depth (m)', 'Euphotic zone depth (m)', 'Sampling depth (m)', 'Depth-resolved chl a (mg m^-3)', 'Depth-integrated chl a (mg m^-2)', 'Total Chl a (mg m^-2)', 'Pbopt (mg C mg Chla-1 h-1)', 'Depth-resolved primary production (mg C m^-3 day^-1)', 'Depth-integrated primary production (mg C m^-2 day^-1)', 'Production to biomass ratio (mg C day-1 / mg Chl a)', 'Bottom depth (m)', 'Distance from coastline (Km)', 'Max sampling depth (m)', 'Max production depth (m)']

Sorted by: ['Year', 'Month', 'Day of the year'] (most recent year first)

Filtered dataset shape: (37722, 23)
Year range in most recent 50 rows: 2016 to 2017
Most recent year: 2017

=== Last 50 Rows - Sorted by Year (Most Recent First) ===
Showing the 50 most recent records

,Date,Year,Month,Day of the year,Latitude,Longitude,SST (°C),surface PAR (E m^-2 day^-1),Day length (h),Mixed Layer Depth (m),Euphotic zone depth (m),Sampling depth (m),Depth-resolved chl a (mg m^-3),Depth-integrated chl a (mg m^-2),Total Chl a (mg m^-2),Pbopt (mg C mg Chla-1 h-1),Depth-resolved primary production (mg C m^-3 day^-1),Depth-integrated primary production (mg C m^-2 day^-1),Production to biomass ratio (mg C day-1 / mg Chl a),Bottom depth (m),Distance from coastline (Km),Max sampling depth (m),Max production depth (m)
14836,15/08/2017,2017,8,227,34.2733,-120.0183,19.0000,57.5100,13.3005,21,18.1598,2.0000,6.1600,108.1550,101.0517,6.6000,172.7000,1666.4100,15.4076,582,20.3230,23.0000,5.0000
14837,15/08/2017,2017,8,227,34.2733,-120.0183,19.0000,57.5100,13.3005,21,18.1598,5.0000,9.6400,108.1550,101.0517,6.6000,189.7000,1666.4100,15.4076,582,20.3230,23.0000,5.0000
14838,15/08/2017,2017,8,227,34.2733,-120.0183,19.0000,57.5100,13.3005,21,18.1598,6.0000,10.4600,108.1550,101.0517,6.6000,156.3000,1666.4100,15.4076,582,20.3230,23.0000,5.0000
14839,15/08/2017,2017,8,227,34.2733,-120.0183,19.0000,57.5100,13.3005,21,18.1598,10.0000,4.7500,108.1550,101.0517,6.6000,41.3000,1666.4100,15.4076,582,20.3230,23.0000,5.0000
14840,15/08/2017,2017,8,227,34.2733,-120.0183,19.0000,57.5100,13.3005,21,18.1598,19.0000,1.3400,108.1550,101.0517,6.6000,3.4000,1666.4100,15.4076,582,20.3230,23.0000,5.0000
14841,15/08/2017,2017,8,227,34.2733,-120.0183,19.0000,57.5100,13.3005,21,18.1598,23.0000,0.7900,108.1550,101.0517,6.6000,0.6300,1666.4100,15.4076,582,20.3230,23.0000,5.0000
14820,14/08/2017,2017,8,226,35.0883,-120.7750,15.0600,43.4400,13.3732,20,16.3429,2.0000,8.1400,126.5400,116.3892,5.7000,220.9000,2234.5000,17.6584,73,6.9512,18.0000,4.0000
14821,14/08/2017,2017,8,226,35.0883,-120.7750,15.0600,43.4400,13.3732,20,16.3429,4.0000,7.8900,126.5400,116.3892,5.7000,232.2000,2234.5000,17.6584,73,6.9512,18.0000,4.0000
14822,14/08/2017,2017,8,226,35.0883,-120.7750,15.0600,43.4400,13.3732,20,16.3429,5.0000,8.7300,126.5400,116.3892,5.7000,202.7000,2234.5000,17.6584,73,6.9512,18.0000,4.0000
14823,14/08/2017,2017,8,226,35.0883,-120.7750,15.0600,43.4400,13.3732,20,16.3429,10.0000,8.9500,126.5400,116.3892,5.7000,93.2000,2234.5000,17.6584,73,6.9512,18.0000,4.0000



=== Summary Statistics for Key Production Columns ===


,Year,Month,Day of the year,Latitude,Longitude,SST (°C),surface PAR (E m^-2 day^-1),Day length (h),Mixed Layer Depth (m),Euphotic zone depth (m),Sampling depth (m),Depth-resolved chl a (mg m^-3),Depth-integrated chl a (mg m^-2),Total Chl a (mg m^-2),Pbopt (mg C mg Chla-1 h-1),Depth-resolved primary production (mg C m^-3 day^-1),Depth-integrated primary production (mg C m^-2 day^-1),Production to biomass ratio (mg C day-1 / mg Chl a),Bottom depth (m),Distance from coastline (Km),Max sampling depth (m),Max production depth (m)
count,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000,37722.0000
mean,1990.4112,5.8772,162.9310,29.5984,-65.8258,16.7979,32.5509,12.5998,37.9041,53.4116,28.9839,1.2523,41.7261,35.1436,5.2990,32.3350,865.7561,28.0695,1758.8310,178.2200,60.2525,11.4191
std,13.1693,3.1722,96.4534,21.6631,75.1616,6.0477,15.4984,2.4265,46.0709,24.1734,35.5830,3.0035,73.0256,29.9512,1.0475,96.9794,1440.2529,58.7901,1794.6187,249.0367,39.8910,13.7118
min,1958.0000,1.0000,1.0000,-77.7400,-188.5000,-1.7400,1.3800,7.9336,1.0000,6.6022,0.0000,0.0000,0.0750,5.3676,1.1300,0.0000,3.3000,0.1464,2.0000,0.4017,1.0000,0.0000
25%,1980.0000,3.0000,86.0000,21.0625,-120.9217,13.8600,14.5100,11.2641,18.0000,33.7132,5.0000,0.1600,17.3162,16.4774,5.1500,1.6000,215.6000,9.0162,77.0000,22.1945,29.5000,2.0000
50%,1988.0000,5.0000,148.0000,33.3000,-73.5000,15.0000,31.7500,12.1034,29.0000,49.9746,16.3500,0.4400,25.9150,26.0146,5.2700,6.2000,469.5950,17.7586,1146.0000,97.0917,49.0000,7.0000
75%,2002.0000,8.0000,242.0000,40.8000,-17.5000,19.9200,46.8200,13.4820,46.0000,70.2591,40.0000,1.2000,43.8750,44.0931,5.9700,26.2650,963.1500,33.8149,3828.0000,207.5350,88.0000,15.0000
max,2017.0000,12.0000,364.0000,80.7700,172.8013,30.1000,63.7700,24.0000,524.0000,122.2372,800.0000,119.0800,2337.0000,392.2643,6.6300,3080.0000,30189.7550,3402.6668,5892.0000,2078.1300,250.0000,107.0000


In [13]:
# ====== Comprehensive Data Cleaning for Phytoplankton Dataset ======
# Clean, transform, and save the dataset

import pandas as pd
import numpy as np
import os
from datetime import datetime

print("=== Starting Comprehensive Data Cleaning ===\n")

# Start with the original dataset
df_clean = df_phytoplankton.copy()

print(f"Original dataset shape: {df_clean.shape}")

# ====== STEP 1: Create unified date column ======
print("\n1. Creating unified date column...")
if all(col in df_clean.columns for col in ['Year', 'Month']):
    # Create date from Year, Month, and Day of the year
    if 'Day of the year' in df_clean.columns:
        # Convert day of year to actual day/month
        df_clean['date'] = pd.to_datetime(
            df_clean['Year'].astype(str) + '-' + 
            df_clean['Month'].astype(str).str.zfill(2) + '-01',
            errors='coerce'
        )
        # Add days from day of year
        df_clean['date'] = df_clean['date'] + pd.to_timedelta(
            (df_clean['Day of the year'] - 1).fillna(0), unit='D'
        )
    elif 'Date' in df_clean.columns:
        df_clean['date'] = pd.to_datetime(df_clean['Date'], errors='coerce')
    else:
        # Use Year and Month only (set to first day of month)
        df_clean['date'] = pd.to_datetime(
            df_clean['Year'].astype(str) + '-' + 
            df_clean['Month'].astype(str).str.zfill(2) + '-01',
            errors='coerce'
        )
    
    # Remove old date columns
    date_cols_to_remove = ['Date', 'Year', 'Month', 'Day of the year']
    for col in date_cols_to_remove:
        if col in df_clean.columns:
            df_clean = df_clean.drop(col, axis=1)
    print(f"   ✓ Created unified 'date' column")
else:
    print("   ⚠ Warning: Could not create date column - missing Year/Month")

# ====== STEP 2: Rename columns to more logical names ======
print("\n2. Renaming columns to logical names...")

# Create a mapping dictionary for column renaming
column_rename_map = {
    # Location
    'Latitude': 'latitude',
    'Longitude': 'longitude',
    
    # Environmental parameters
    'SST (°C)': 'sea_surface_temp_c',
    'SST_flag': 'sst_quality_flag',
    'surface PAR (E m^-2 day^-1)': 'surface_par',
    'PAR_flag': 'par_quality_flag',
    'Day length (h)': 'day_length_hours',
    
    # Depth measurements
    'Bottom depth (m)': 'bottom_depth_m',
    'Bottom depth sd (m)': 'bottom_depth_sd_m',
    'Mixed Layer Depth (m)': 'mixed_layer_depth_m',
    'Euphotic zone depth (m)': 'euphotic_zone_depth_m',
    'Sampling depth (m)': 'sampling_depth_m',
    'Max sampling depth (m)': 'max_sampling_depth_m',
    'Max production depth (m)': 'max_production_depth_m',
    'Distance from coastline (Km)': 'distance_from_coast_km',
    
    # Chlorophyll measurements
    'Depth-resolved chl a (mg m^-3)': 'chl_a_resolved_mg_m3',
    'Depth-integrated chl a (mg m^-2)': 'chl_a_integrated_mg_m2',
    'Total Chl a (mg m^-2)': 'chl_a_total_mg_m2',
    
    # Primary production
    'Depth-resolved primary production (mg C m^-3 day^-1)': 'primary_prod_resolved_mg_c_m3_day',
    'Depth-integrated primary production (mg C m^-2 day^-1)': 'primary_prod_integrated_mg_c_m2_day',
    'Production to biomass ratio (mg C day-1 / mg Chl a)': 'prod_to_biomass_ratio',
    'Pbopt (mg C mg Chla-1 h-1)': 'max_production_rate_pbopt',
    
    # Other identifiers
    'Event': 'event_name',
    'Short reference': 'reference',
    'Paper doi': 'paper_doi',
    'Data doi/data link': 'data_link',
    'Profile number': 'profile_number',
    'Count': 'record_id',
    
    # Magnitude categories (if they exist)
    'hemisphere': 'hemisphere',
    'Northern hemisphere season': 'season',
}

# Apply renaming (only for columns that exist)
columns_to_rename = {k: v for k, v in column_rename_map.items() if k in df_clean.columns}
df_clean = df_clean.rename(columns=columns_to_rename)
print(f"   ✓ Renamed {len(columns_to_rename)} columns")

# ====== STEP 3: Remove unnecessary columns ======
print("\n3. Removing unnecessary columns...")
cols_before = len(df_clean.columns)

# Remove columns that are mostly flags or metadata we don't need
unnecessary_cols = [
    'id',  # If exists and redundant
    'record_id',  # Keep only if needed, but usually redundant with index
]

# Remove magnitude category columns (they're derived, not original data)
magnitude_cols = [col for col in df_clean.columns if 'magnitude' in col.lower()]
unnecessary_cols.extend(magnitude_cols)

# Remove columns that are entirely empty or have very low data coverage (<5%)
for col in df_clean.columns:
    if df_clean[col].notna().sum() / len(df_clean) < 0.05:
        unnecessary_cols.append(col)

# Remove duplicates from list
unnecessary_cols = list(set([col for col in unnecessary_cols if col in df_clean.columns]))
df_clean = df_clean.drop(columns=unnecessary_cols, errors='ignore')
cols_removed = cols_before - len(df_clean.columns)
print(f"   ✓ Removed {cols_removed} unnecessary columns")

# ====== STEP 4: Clean null values ======
print("\n4. Cleaning null values...")
nulls_before = df_clean.isnull().sum().sum()

# Strategy: 
# - For numeric columns: fill with median (less sensitive to outliers than mean)
# - For categorical: fill with mode or 'unknown'
# - For critical columns: keep nulls or use forward fill if time series

# Fill numeric columns with median
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if col != 'date':  # Don't fill date column
        median_val = df_clean[col].median()
        if pd.notna(median_val):
            df_clean[col] = df_clean[col].fillna(median_val)

# Fill categorical columns with mode or 'unknown'
categorical_cols = df_clean.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if col != 'date':
        mode_val = df_clean[col].mode()
        if len(mode_val) > 0:
            df_clean[col] = df_clean[col].fillna(mode_val[0])
        else:
            df_clean[col] = df_clean[col].fillna('unknown')

nulls_after = df_clean.isnull().sum().sum()
nulls_cleaned = nulls_before - nulls_after
print(f"   ✓ Cleaned {nulls_cleaned:,} null values ({nulls_before:,} → {nulls_after:,})")

# ====== STEP 5: Remove duplicate rows ======
print("\n5. Removing duplicate rows...")
rows_before = len(df_clean)
df_clean = df_clean.drop_duplicates()
rows_removed = rows_before - len(df_clean)
print(f"   ✓ Removed {rows_removed} duplicate rows")

# ====== STEP 6: Remove rows with critical missing data ======
print("\n6. Removing rows with critical missing data...")
rows_before = len(df_clean)

# Remove rows where essential columns are missing
critical_cols = ['latitude', 'longitude', 'date']
if all(col in df_clean.columns for col in critical_cols):
    df_clean = df_clean.dropna(subset=critical_cols)
    rows_removed = rows_before - len(df_clean)
    print(f"   ✓ Removed {rows_removed} rows with missing critical data (location/date)")

# ====== STEP 7: Sort by date ======
print("\n7. Sorting by date...")
if 'date' in df_clean.columns:
    df_clean = df_clean.sort_values('date', ascending=False, na_position='last')
    print(f"   ✓ Sorted by date (most recent first)")

# ====== STEP 8: Reset index ======
df_clean = df_clean.reset_index(drop=True)

# ====== STEP 9: Final data quality check ======
print("\n8. Final data quality check...")
print(f"   Final shape: {df_clean.shape}")
print(f"   Columns: {len(df_clean.columns)}")
print(f"   Rows: {len(df_clean):,}")
print(f"   Remaining nulls: {df_clean.isnull().sum().sum():,}")

# Show column list
print(f"\n   Cleaned columns ({len(df_clean.columns)}):")
for i, col in enumerate(df_clean.columns, 1):
    null_pct = (df_clean[col].isnull().sum() / len(df_clean) * 100)
    print(f"   {i:2d}. {col:40s} (nulls: {null_pct:.1f}%)")

# ====== STEP 10: Save to CSV ======
print("\n9. Saving cleaned dataset to CSV...")
output_filename = 'phytoplankton_production_cleaned.csv'
df_clean.to_csv(output_filename, index=False, encoding='utf-8')

# Get file size
file_size = os.path.getsize(output_filename)
file_size_mb = file_size / (1024 * 1024)
print(f"   ✓ Saved to: {output_filename}")
print(f"   File size: {file_size_mb:.2f} MB ({file_size:,} bytes)")

# Display sample of cleaned data
print(f"\n=== Sample of Cleaned Data (First 10 rows) ===")
display(df_clean.head(10))

print(f"\n=== Data Cleaning Complete! ===")
print(f"Original: {df_phytoplankton.shape[0]:,} rows × {df_phytoplankton.shape[1]} columns")
print(f"Cleaned:  {df_clean.shape[0]:,} rows × {df_clean.shape[1]} columns")
print(f"Saved to: {output_filename}")


=== Starting Comprehensive Data Cleaning ===

Original dataset shape: (37722, 49)

1. Creating unified date column...
   ✓ Created unified 'date' column

2. Renaming columns to logical names...
   ✓ Renamed 30 columns

3. Removing unnecessary columns...
   ✓ Removed 16 unnecessary columns

4. Cleaning null values...
   ✓ Cleaned 21,088 null values (21,088 → 0)

5. Removing duplicate rows...
   ✓ Removed 0 duplicate rows

6. Removing rows with critical missing data...
   ✓ Removed 0 rows with missing critical data (location/date)

7. Sorting by date...
   ✓ Sorted by date (most recent first)

8. Final data quality check...
   Final shape: (37722, 30)
   Columns: 30
   Rows: 37,722
   Remaining nulls: 0

   Cleaned columns (30):
    1. event_name                               (nulls: 0.0%)
    2. reference                                (nulls: 0.0%)
    3. paper_doi                                (nulls: 0.0%)
    4. data_link                                (nulls: 0.0%)
    5. profile_

,event_name,reference,paper_doi,data_link,profile_number,latitude,longitude,day_length_hours,bottom_depth_m,bottom_depth_sd_m,...,max_production_rate_pbopt,chl_a_resolved_mg_m3,chl_a_integrated_mg_m2,chl_a_total_mg_m2,primary_prod_resolved_mg_c_m3_day,primary_prod_integrated_mg_c_m2_day,prod_to_biomass_ratio,hemisphere,season,date
0,CALCOFI,calcofi.org,https://doi.org/10.4319/lo.1997.42.1.0001,http://calcofi.org/downloads/IEH/,2309,34.273333,-120.018333,13.300491,582,2.249829,...,6.6,6.16,108.155,101.051665,172.70,1666.41,15.407609,northern,summer,2018-03-15
1,CALCOFI,calcofi.org,https://doi.org/10.4319/lo.1997.42.1.0001,http://calcofi.org/downloads/IEH/,2309,34.273333,-120.018333,13.300491,582,2.249829,...,6.6,0.79,108.155,101.051665,0.63,1666.41,15.407609,northern,summer,2018-03-15
2,CALCOFI,calcofi.org,https://doi.org/10.4319/lo.1997.42.1.0001,http://calcofi.org/downloads/IEH/,2309,34.273333,-120.018333,13.300491,582,2.249829,...,6.6,10.46,108.155,101.051665,156.30,1666.41,15.407609,northern,summer,2018-03-15
3,CALCOFI,calcofi.org,https://doi.org/10.4319/lo.1997.42.1.0001,http://calcofi.org/downloads/IEH/,2309,34.273333,-120.018333,13.300491,582,2.249829,...,6.6,4.75,108.155,101.051665,41.30,1666.41,15.407609,northern,summer,2018-03-15
4,CALCOFI,calcofi.org,https://doi.org/10.4319/lo.1997.42.1.0001,http://calcofi.org/downloads/IEH/,2309,34.273333,-120.018333,13.300491,582,2.249829,...,6.6,1.34,108.155,101.051665,3.40,1666.41,15.407609,northern,summer,2018-03-15
5,CALCOFI,calcofi.org,https://doi.org/10.4319/lo.1997.42.1.0001,http://calcofi.org/downloads/IEH/,2309,34.273333,-120.018333,13.300491,582,2.249829,...,6.6,9.64,108.155,101.051665,189.70,1666.41,15.407609,northern,summer,2018-03-15
6,CALCOFI,calcofi.org,https://doi.org/10.4319/lo.1997.42.1.0001,http://calcofi.org/downloads/IEH/,2306,35.088333,-120.775000,13.373152,73,4.012327,...,5.7,1.48,126.540,116.389185,2.40,2234.50,17.658448,northern,summer,2018-03-14
7,CALCOFI,calcofi.org,https://doi.org/10.4319/lo.1997.42.1.0001,http://calcofi.org/downloads/IEH/,2306,35.088333,-120.775000,13.373152,73,4.012327,...,5.7,8.95,126.540,116.389185,93.20,2234.50,17.658448,northern,summer,2018-03-14
8,CALCOFI,calcofi.org,https://doi.org/10.4319/lo.1997.42.1.0001,http://calcofi.org/downloads/IEH/,2306,35.088333,-120.775000,13.373152,73,4.012327,...,5.7,8.73,126.540,116.389185,202.70,2234.50,17.658448,northern,summer,2018-03-14
9,CALCOFI,calcofi.org,https://doi.org/10.4319/lo.1997.42.1.0001,http://calcofi.org/downloads/IEH/,2306,35.088333,-120.775000,13.373152,73,4.012327,...,5.7,7.89,126.540,116.389185,232.20,2234.50,17.658448,northern,summer,2018-03-14



=== Data Cleaning Complete! ===
Original: 37,722 rows × 49 columns
Cleaned:  37,722 rows × 30 columns
Saved to: phytoplankton_production_cleaned.csv


In [14]:
import pandas as pd
import numpy as np

print("=== Cleaning CalCOFI Data for Algae Farming Advisory ===\n")

# Load your CalCOFI dataset
# df_calcofi = pd.read_csv('your_calcofi_file.csv')

# ====== COLUMNS TO KEEP ======
# These are the only columns useful for algae farming advisory

columns_to_keep = [
    # ESSENTIAL - Location & Time
    'latitude',
    'longitude', 
    'date',
    
    # ESSENTIAL - Environmental Parameters for Farming
    'sea_surface_temp_c',          # Temperature is critical for species selection
    'surface_par',                  # Light availability
    'day_length_hours',             # Photoperiod
    'season',                       # Seasonal patterns
    'hemisphere',                   # Growing season timing
    
    # USEFUL - Location Context
    'distance_from_coast_km',       # How far from shore (farming usually <5km)
    
    # USEFUL - Water Column (Optional - keep if you want depth info)
    'euphotic_zone_depth_m',        # Light penetration depth
    'sampling_depth_m',             # Where measurement was taken
    
    # USEFUL - Baseline Productivity Proxy (Optional)
    'primary_prod_integrated_mg_c_m2_day',  # Natural productivity = nutrient proxy
]

# ====== COLUMNS TO REMOVE ======
# These are NOT useful for algae farming advisory

columns_to_remove = [
    # Metadata - not needed for analysis
    'event_name',
    'reference', 
    'paper_doi',
    'data_link',
    'profile_number',
    
    # Quality flags - not needed after data validation
    'sst_quality_flag',
    'par_quality_flag',
    
    # Deep water metrics - irrelevant for farming (CalCOFI is 20km offshore, 500m+ deep)
    'bottom_depth_m',              # 582m - way too deep for farming
    'bottom_depth_sd_m',
    'mixed_layer_depth_m',         # Oceanographic research metric
    'max_sampling_depth_m',
    'max_production_depth_m',
    
    # Wild phytoplankton chlorophyll - not farm yields
    'chl_a_resolved_mg_m3',
    'chl_a_integrated_mg_m2', 
    'chl_a_total_mg_m2',
    
    # Research-specific productivity metrics - too detailed for advisory
    'primary_prod_resolved_mg_c_m3_day',  # Keep only integrated version
    'prod_to_biomass_ratio',
    'max_production_rate_pbopt',
]

# ====== APPLY CLEANING ======

def clean_calcofi_for_advisory(df):
    """
    Clean CalCOFI dataset - keep only columns useful for algae farming advisory
    """
    df_clean = df.copy()
    
    print(f"Original shape: {df_clean.shape}")
    print(f"Original columns: {len(df_clean.columns)}\n")
    
    # Keep only useful columns (that exist in the dataset)
    existing_cols_to_keep = [col for col in columns_to_keep if col in df_clean.columns]
    df_clean = df_clean[existing_cols_to_keep]
    
    print(f"✓ Kept {len(existing_cols_to_keep)} useful columns:")
    for i, col in enumerate(existing_cols_to_keep, 1):
        print(f"  {i:2d}. {col}")
    
    # Remove rows with missing critical data
    print(f"\nRemoving rows with missing critical data...")
    rows_before = len(df_clean)
    critical_cols = ['latitude', 'longitude', 'date', 'sea_surface_temp_c']
    critical_cols_exist = [col for col in critical_cols if col in df_clean.columns]
    df_clean = df_clean.dropna(subset=critical_cols_exist)
    rows_removed = rows_before - len(df_clean)
    print(f"✓ Removed {rows_removed} rows with missing critical data")
    
    # Remove duplicates
    print(f"\nRemoving duplicates...")
    rows_before = len(df_clean)
    df_clean = df_clean.drop_duplicates()
    rows_removed = rows_before - len(df_clean)
    print(f"✓ Removed {rows_removed} duplicate rows")
    
    # Sort by date (most recent first)
    if 'date' in df_clean.columns:
        df_clean['date'] = pd.to_datetime(df_clean['date'], errors='coerce')
        df_clean = df_clean.sort_values('date', ascending=False)
        print(f"✓ Sorted by date (most recent first)")
    
    # Reset index
    df_clean = df_clean.reset_index(drop=True)
    
    print(f"\n=== Final Cleaned Dataset ===")
    print(f"Shape: {df_clean.shape}")
    print(f"Date range: {df_clean['date'].min()} to {df_clean['date'].max()}")
    print(f"Location range: Lat {df_clean['latitude'].min():.2f} to {df_clean['latitude'].max():.2f}")
    print(f"                Long {df_clean['longitude'].min():.2f} to {df_clean['longitude'].max():.2f}")
    
    # Data completeness check
    print(f"\nData completeness:")
    for col in df_clean.columns:
        completeness = (1 - df_clean[col].isnull().sum() / len(df_clean)) * 100
        print(f"  {col:40s}: {completeness:5.1f}% complete")
    
    return df_clean


# ====== USAGE EXAMPLE ======
# Uncomment and use with your actual data:

# df_calcofi = pd.read_csv('calcofi_data.csv')
# df_calcofi_clean = clean_calcofi_for_advisory(df_calcofi)
# df_calcofi_clean.to_csv('calcofi_cleaned_for_advisory.csv', index=False)
# print(f"\n✓ Saved to: calcofi_cleaned_for_advisory.csv")

# Display sample
# print("\nSample of cleaned data:")
# print(df_calcofi_clean.head(10))

=== Cleaning CalCOFI Data for Algae Farming Advisory ===



In [21]:

df_calcofi = pd.read_csv('phytoplankton_production_cleaned.csv')
df_calcofi_clean = clean_calcofi_for_advisory(df_calcofi)
df_calcofi_clean.to_csv('calcofi_cleaned_for_advisory.csv', index=False)
print(f"\n✓ Saved to: calcofi_cleaned_for_advisory.csv")


print("\nSample of cleaned data:")
print(df_calcofi_clean.head(10))

Original shape: (37722, 30)
Original columns: 30

✓ Kept 12 useful columns:
   1. latitude
   2. longitude
   3. date
   4. sea_surface_temp_c
   5. surface_par
   6. day_length_hours
   7. season
   8. hemisphere
   9. distance_from_coast_km
  10. euphotic_zone_depth_m
  11. sampling_depth_m
  12. primary_prod_integrated_mg_c_m2_day

Removing rows with missing critical data...
✓ Removed 0 rows with missing critical data

Removing duplicates...
✓ Removed 337 duplicate rows
✓ Sorted by date (most recent first)

=== Final Cleaned Dataset ===
Shape: (37385, 12)
Date range: 1958-11-04 00:00:00 to 2018-03-15 00:00:00
Location range: Lat -77.74 to 80.77
                Long -188.50 to 172.80

Data completeness:
  latitude                                : 100.0% complete
  longitude                               : 100.0% complete
  date                                    : 100.0% complete
  sea_surface_temp_c                      : 100.0% complete
  surface_par                             : 1

In [23]:
import pandas as pd
import numpy as np
import ee
from datetime import datetime

print("=== CalCOFI + MODIS Seasonal Merge (Demo-Ready for Judges) ===\n")

# Initialize Earth Engine
# ee.Initialize()


def extract_modis_par_seasonal(df_calcofi, start_date, end_date, max_points_per_batch=1000):
    """
    BEST FOR DEMO: Merge using SEASONS instead of exact dates
    
    Why this is better for judges:
    - Simple to explain: "Winter PAR, Spring PAR, Summer PAR, Fall PAR"
    - Scientifically sound: Algae farmers care about seasons, not exact days
    - Fast and reliable: No complex date matching
    - Handles Earth Engine limits properly
    
    Seasons:
    - Winter: Dec, Jan, Feb (months 12, 1, 2)
    - Spring: Mar, Apr, May (months 3, 4, 5)
    - Summer: Jun, Jul, Aug (months 6, 7, 8)
    - Fall: Sep, Oct, Nov (months 9, 10, 11)
    """
    
    print("🌍 Loading MODIS satellite data (2010-2020)...")
    modis = ee.ImageCollection('MODIS/062/MCD18C2') \
              .filterDate(start_date, end_date) \
              .select('GMT_1200_PAR')
    
    print(f"✓ Loaded {modis.size().getInfo()} daily satellite images")
    
    # Define seasons
    seasons = {
        'Winter': [12, 1, 2],
        'Spring': [3, 4, 5],
        'Summer': [6, 7, 8],
        'Fall': [9, 10, 11]
    }
    
    print("\n📊 Creating seasonal PAR composites...")
    print("   (Averaging 10 years of satellite data for each season)\n")
    
    seasonal_par = {}
    for season_name, months in seasons.items():
        # Filter to months in this season and take mean
        season_filter = ee.Filter.Or(*[
            ee.Filter.calendarRange(month, month, 'month') 
            for month in months
        ])
        seasonal_image = modis.filter(season_filter).mean()
        seasonal_par[season_name] = seasonal_image
        print(f"   ✓ {season_name:6s} composite ready (months {months})")
    
    # Add season column to dataframe
    df_calcofi['date_parsed'] = pd.to_datetime(df_calcofi['date'])
    df_calcofi['month'] = df_calcofi['date_parsed'].dt.month
    
    def get_season(month):
        if month in [12, 1, 2]:
            return 'Winter'
        elif month in [3, 4, 5]:
            return 'Spring'
        elif month in [6, 7, 8]:
            return 'Summer'
        else:
            return 'Fall'
    
    df_calcofi['season'] = df_calcofi['month'].apply(get_season)
    
    print(f"\n🎯 Sampling {len(df_calcofi):,} locations by season...")
    print(f"   (Processing in batches of {max_points_per_batch} to respect Earth Engine limits)\n")
    
    # Process each season
    all_par_values = {}
    
    for season_name in ['Winter', 'Spring', 'Summer', 'Fall']:
        season_data = df_calcofi[df_calcofi['season'] == season_name]
        
        if len(season_data) == 0:
            continue
        
        print(f"   {season_name}: {len(season_data):,} locations", end='')
        
        # Split into batches to avoid Earth Engine 5000 element limit
        num_batches = (len(season_data) + max_points_per_batch - 1) // max_points_per_batch
        
        if num_batches > 1:
            print(f" (split into {num_batches} batches)")
        else:
            print()
        
        season_par_values = {}
        
        for batch_idx in range(num_batches):
            start_idx = batch_idx * max_points_per_batch
            end_idx = min((batch_idx + 1) * max_points_per_batch, len(season_data))
            batch = season_data.iloc[start_idx:end_idx]
            
            if num_batches > 1:
                print(f"      Batch {batch_idx + 1}/{num_batches}: {len(batch)} points...", end='')
            
            # Create points for this batch
            features = []
            for idx, row in batch.iterrows():
                point = ee.Geometry.Point([row['longitude'], row['latitude']])
                features.append(ee.Feature(point, {'row_idx': int(idx)}))
            
            points = ee.FeatureCollection(features)
            
            # Sample seasonal composite
            par_image = seasonal_par[season_name]
            
            try:
                sampled = par_image.reduceRegions(
                    collection=points,
                    reducer=ee.Reducer.first(),
                    scale=5600
                )
                
                sampled_list = sampled.getInfo()['features']
                
                for feature in sampled_list:
                    props = feature['properties']
                    row_idx = props.get('row_idx')
                    par_value = props.get('GMT_1200_PAR')
                    season_par_values[row_idx] = par_value if par_value is not None else np.nan
                
                if num_batches > 1:
                    print(" ✓")
                
            except Exception as e:
                print(f" ⚠ Error: {e}")
                for idx in batch.index:
                    season_par_values[idx] = np.nan
        
        # Store results for this season
        all_par_values.update(season_par_values)
        
        if num_batches == 1:
            print(f"      ✓ Done")
    
    # Add PAR values to dataframe
    df_calcofi['modis_par_seasonal'] = df_calcofi.index.map(lambda idx: all_par_values.get(idx, np.nan))
    
    # Calculate statistics
    matched = df_calcofi['modis_par_seasonal'].notna().sum()
    match_rate = (matched / len(df_calcofi)) * 100
    
    print(f"\n✅ MERGE COMPLETE!")
    print(f"   Total locations: {len(df_calcofi):,}")
    print(f"   Successfully matched: {matched:,} ({match_rate:.1f}%)")
    print(f"   Missing data: {len(df_calcofi) - matched:,} ({100-match_rate:.1f}%)")
    
    # Clean up temporary columns
    df_calcofi = df_calcofi.drop(['date_parsed', 'month'], axis=1)
    
    return df_calcofi


def create_demo_summary(df_merged):
    """
    Create a clear summary for judges
    Shows the merge worked and makes sense
    """
    
    print("\n" + "="*70)
    print("📈 DEMO SUMMARY FOR JUDGES")
    print("="*70)
    
    print("\n1️⃣  WHAT WE DID:")
    print("   Combined two datasets:")
    print("   • CalCOFI: Ship-based ocean measurements (ground truth)")
    print("   • MODIS: Satellite observations (global coverage)")
    print("   Result: Environmental database for algae farming advisory")
    
    print("\n2️⃣  HOW WE MERGED:")
    print("   Matched by: Location + Season")
    print("   - CalCOFI gives us exact locations along California coast")
    print("   - MODIS satellite provides light data (PAR) for those locations")
    print("   - Grouped by season (Winter/Spring/Summer/Fall)")
    
    print("\n3️⃣  DATA QUALITY:")
    total = len(df_merged)
    matched = df_merged['modis_par_seasonal'].notna().sum()
    match_pct = (matched / total) * 100
    
    print(f"   Total measurements: {total:,}")
    print(f"   Successfully merged: {matched:,} ({match_pct:.1f}%)")
    
    # Show by season
    print("\n   Breakdown by season:")
    for season in ['Winter', 'Spring', 'Summer', 'Fall']:
        season_data = df_merged[df_merged['season'] == season]
        season_matched = season_data['modis_par_seasonal'].notna().sum()
        season_pct = (season_matched / len(season_data) * 100) if len(season_data) > 0 else 0
        print(f"      {season:6s}: {len(season_data):5,} points ({season_matched:5,} matched, {season_pct:.1f}%)")
    
    print("\n4️⃣  VALIDATION:")
    if 'surface_par' in df_merged.columns:
        df_compare = df_merged[
            df_merged['surface_par'].notna() & 
            df_merged['modis_par_seasonal'].notna()
        ]
        
        if len(df_compare) > 0:
            correlation = df_compare['surface_par'].corr(df_compare['modis_par_seasonal'])
            print(f"   Correlation (ship vs satellite): {correlation:.3f}")
            
            if correlation > 0.7:
                print("   ✓ EXCELLENT - Satellite data matches ship measurements!")
            elif correlation > 0.5:
                print("   ✓ GOOD - Datasets agree reasonably well")
            else:
                print("   ⚠ Moderate agreement - seasonal averaging expected")
            
            calcofi_mean = df_compare['surface_par'].mean()
            modis_mean = df_compare['modis_par_seasonal'].mean()
            print(f"   Average PAR - Ship: {calcofi_mean:.1f}, Satellite: {modis_mean:.1f}")
        else:
            print("   No overlapping measurements for direct comparison")
    
    print("\n5️⃣  SAMPLE DATA:")
    print("\n   Example merged records:")
    sample = df_merged[df_merged['modis_par_seasonal'].notna()].head(5)
    
    if len(sample) > 0:
        print("\n   Location        Season    Temp(°C)  Ship-PAR  Satellite-PAR")
        print("   " + "-"*62)
        for _, row in sample.iterrows():
            lat = row.get('latitude', 0)
            lon = row.get('longitude', 0)
            season = row.get('season', 'N/A')
            temp = row.get('sea_surface_temp_c', np.nan)
            ship_par = row.get('surface_par', np.nan)
            sat_par = row.get('modis_par_seasonal', np.nan)
            
            print(f"   {lat:6.2f},{lon:7.2f}  {season:6s}  {temp:6.1f}    {ship_par:7.1f}      {sat_par:7.1f}")
    
    print("\n6️⃣  BUSINESS VALUE:")
    print("   This merged dataset enables:")
    print("   ✓ Predict light availability at any coastal location")
    print("   ✓ Identify best seasons for algae farming")
    print("   ✓ Match environmental conditions to optimal species")
    print("   ✓ Provide data-driven investment recommendations")
    
    print("\n" + "="*70)
    print("✅ READY FOR DEMO")
    print("="*70 + "\n")


# ====== MAIN EXECUTION ======

def run_demo_merge(calcofi_file):
    """
    Complete workflow for judges demo
    """
    
    print("🚀 Starting CalCOFI + MODIS Seasonal Merge\n")
    
    # Load data
    print("📂 Loading CalCOFI data...")
    df_calcofi = pd.read_csv(calcofi_file)
    print(f"✓ Loaded {len(df_calcofi):,} CalCOFI measurements\n")
    
    # Run merge
    df_merged = extract_modis_par_seasonal(
        df_calcofi, 
        start_date='2010-01-01', 
        end_date='2020-12-31',
        max_points_per_batch=1000  # Stay well under 5000 limit
    )
    
    # Save results
    output_file = 'calcofi_with_seasonal_par.csv'
    df_merged.to_csv(output_file, index=False)
    print(f"\n💾 Saved to: {output_file}")
    
    # Create demo summary
    create_demo_summary(df_merged)
    
    return df_merged


# ====== USAGE ======

# Run this for your demo:
# df_result = run_demo_merge('calcofi_cleaned_for_advisory.csv')

print("✅ Code ready!")
print("Run: df_result = run_demo_merge('calcofi_cleaned_for_advisory.csv')")

=== CalCOFI + MODIS Seasonal Merge (Demo-Ready for Judges) ===

✅ Code ready!
Run: df_result = run_demo_merge('calcofi_cleaned_for_advisory.csv')


In [ ]:

df_result = run_demo_merge('calcofi_cleaned_for_advisory.csv')

🚀 Starting CalCOFI + MODIS Seasonal Merge

📂 Loading CalCOFI data...
✓ Loaded 37,385 CalCOFI measurements

🌍 Loading MODIS satellite data (2010-2020)...
✓ Loaded 3961 daily satellite images

📊 Creating seasonal PAR composites...
   (Averaging 10 years of satellite data for each season)

   ✓ Winter composite ready (months [12, 1, 2])
   ✓ Spring composite ready (months [3, 4, 5])
   ✓ Summer composite ready (months [6, 7, 8])
   ✓ Fall   composite ready (months [9, 10, 11])

🎯 Sampling 37,385 locations by season...
   (Processing in batches of 1000 to respect Earth Engine limits)

   Winter: 6,876 locations (split into 7 batches)
      Batch 1/7: 1000 points... ✓
      Batch 2/7: 1000 points... ✓
      Batch 3/7: 1000 points... ✓
      Batch 4/7: 1000 points... ✓
      Batch 5/7: 1000 points...